In [ ]:
"""###################################################  data_provider.py  ##############################################################"""

import numpy as np
import glob
import collections
import re
import math
import random
import string
import cv2
import copy
import h5py
from types import TracebackType
import os
import sys
from typing import Any, Optional, List, Dict, Type


from .label import Label
from ..sensors.lidar import Lidar
from ..sensors.radar import Radar
from ..sensors.camera import Camera
from ..calibrations.lidar_calib import LidarCalib
from ..calibrations.radar_calib import RadarCalib
from ..calibrations.camera_calib import CameraCalib
from .cbor_handler import CBOR_Handler
from end2end_utilities.class_mapper.class_mapper import ClassMapper


class DataProviderDataSpec:
    """
    structure for DataProviderGenerator input
    """

    def __init__(self, folder: str, type: str, sequence_list: List[str], label_source: str, mapping_file: Optional[str] = None):
        """
        inits DataProviderGenerator

        Args:
            folder:         data root folder
            type:           data format out of ['trainingTool2', 'cbor']
            sequence_list:  name of logs to load
            label_source:   source of the labeling (e.g. 'XC90 Pandora', 'Understand AI', 'Autolabeling-EndReview-FixedLeGO-LOAM')
                            or None if no labels shall be loaded at all
            mapping_file:   path of class mapping file
        """
        if type not in ["trainingTool2", "cbor"]:
            raise ValueError("Data type %s not supported, in DataProviderDataSpec.__init__" % type)

        # store input
        self.folder = folder
        self.type = type
        self.sequence_list = sequence_list
        self.label_source = label_source
        self.mapping_file = mapping_file

        # expand sequence list if all sequences are required
        if not self.sequence_list:
            if self.type == "trainingTool2":
                seq_in_folder_dict = {os.path.splitext(os.path.basename(ff))[0]: ff for ff in glob.glob(os.path.join(self.folder, "Meta", "*.h5"))}
                self.sequence_list = seq_in_folder_dict.keys()
            elif self.type == "cbor":
                seq_in_folder_dict = {os.path.basename(ff).split(".")[0].split("_")[1]: ff for ff in glob.glob(os.path.join(self.folder, "config_*.yml"))}
                self.sequence_list = seq_in_folder_dict.keys()

        # get data versions
        if self.type == "cbor":
            # NOTE: no versioning for cbor data available yet
            self.data_version = [None for log in self.sequence_list]
        elif self.type == "trainingTool2":
            self.data_version = []
            for log in self.sequence_list:
                meta_file = h5py.File(os.path.join(self.folder, "Meta", log + ".h5"), "r")
                self.data_version.append(meta_file["info"].attrs["version"])
                meta_file.close()

    def get_log_specs(self) -> List[Dict[str, str]]:
        """
        get evaluated log list defined by data_spec

        Returns:
            List of dicts holding {log_type: <sequence data type (e.g.: 'cbor')>,
                                   log_folder: <data folder>,
                                   log_name: <list of sequence names>,
                                   data_version: <data version>
                                   label_source: <GT label source>,
                                   mapping_file: <mapping file>
        """
        return [
            {
                "log_type": self.type,
                "log_folder": self.folder,
                "log_name": log,
                "data_version": version,
                "label_source": self.label_source,
                "mapping_file": self.mapping_file,
            }
            for log, version in zip(self.sequence_list, self.data_version)
        ]


"""
structure for file meta data:
    sensor_type:        type of sensor
    sensor_id:          id of sensor
    sensor_name:        name of sensor
    source_type:        type of source (e.g. trainingTool2, etc.)
    folder:             superfolder containing data
    sequence_name:      subfolder containing sequence the file belongs to
    sequence_number:    order position of sequence in complete loaded sequence list
    aligned_sensor_names:  name of aligned sensors per [sensor][sensor id]
    aligned_fids:       frame id within file per [sensor][sensor id] (can alternatively be one number for all)
    aligned_timestamps: timestamp per [sensor][sensor id] and [label] (can alternatively be one number for all)
    aligned_label_ids:  label file idx per [sensor][sensor id] (can alternatively be one number for all)
    aligned_sids:       aligned scan ids from sensor hardware
    aligned_subsensors: subsensor information for artificial fused sensors
    framenumber:        order position of file within sequence
    data_handlers:      data handling structure if needed for data type
    flags:              dict to be used to pass additional information in inherited DataProviders
"""
DataProviderMetaData = collections.namedtuple(
    "DataProviderMetaData",
    "sensor_type "
    "sensor_id "
    "sensor_name "
    "source_type "
    "folder "
    "sequence_name "
    "sequence_number "
    "aligned_sensor_names "
    "aligned_fids "
    "aligned_timestamps "
    "aligned_sids "
    "aligned_subsensors "
    "framenumber "
    "data_handlers "
    "flags",
)


class DataManipulator:
    """
    Basic data manipulator object
    """

    # data_package and data_config need always to be first arguments after self
    def __init__(self, data_package: "DataProvider", data_config: dict):
        """
        dummy for data manipulation function which can be added in derived classes, to change the inserted DataProvider
        Note: this function will not be called again when DataProvider is taken from scratch

        Args:
            data_package:   DataProvider to be manipulated
            data_config:    Additional information from DataProvider generation that might be used at manipulation step
        """
        pass
        # print('Manipulating %s' % data_package.tag)

    def update_cache_independent(self, data_package: "DataProvider", data_config: dict, **args: Any):
        """
        dummy for data manipulations that shall be performed always, even if the DataProvider was taken from cache

        Args:
            data_package:   DataProvider to be manipulated
            data_config:    Additional information from DataProvider generation that might be used at manipulation step
            **args:         dummy for additional arguments, replace by the arguments you use in __init__ function (IMPORTANT)
        """
        pass
        # print('Updating %s' % data_package.tag)

    def draw_2d_overlay(self, data_package: "DataProvider", cam: int, canvas: "vtk.vtkImageCanvasSource2D"):
        """
        dummy for plot functions which can be added in derived classes, to plot class specific overlays on camera images

        Args:
            data_package:   DataProvider holding data to plot
            cam:            holds the camera list id
            canvas:         holds the viewer 2d vtk rendering struct for this camera (see vtk.vtkImageCanvasSource2D)
        """
        pass
        # do not plot anything with this function in base class, use a derived class with overloaded function
        # e.g. plot a red box on image canvas (vtk.vtkImageCanvasSource2D):
        # canvas.SetDrawColor(255.0, 0.0, 0.0, 255.0)
        # canvas.FillBox(10, 100, 10, 100)

    def add_2d_actors(self, data_package: "DataProvider", cam: int, actor_data_list: list, xc: float, yc: float):
        """
        dummy for plot functions which can be added in derived classes, to add class specific actors in 2d view,
        image plane is located between X(right) and Y(down) axes, with top left edge in origin
        (i.e. center at x=width/2, y=height/2, z=0), camera recording this plane is located at x=width/2, y=height/2, z=-height/2,
        thus, horizontal aperture angle=90deg

        Args:
            data_package:       DataProvider holding data to plot
            cam:                holds the camera list id
            actor_data_list:    list to add your plot data to, as tuple (vtk object actors, list with data to be kept in memory as long as data is plotted)
            xc:                 hold image width/2 as center of image plane position
            yc:                 hold image height/2 as center of image plane position
        """

        pass
        # do not plot anything with this function in base class, use a derived class with overloaded function
        # e.g. plot a red cube at origin, so that half of it will be in front of image plane while half will be occluded:
        # import vtk
        # # create cube structure
        # cube = vtk.vtkCubeSource()
        # cube.SetCenter(xc, yc, 0)
        # cube.SetXLength(200)
        # cube.SetYLength(200)
        # cube.SetZLength(200)
        # # create actor
        # cubeActor = vtk.vtkActor()
        # cubeActor.GetProperty().SetRepresentationToWireframe()
        # cubeActor.GetProperty().SetColor(1, 0, 0)
        # cubeActor.GetProperty().LightingOff()
        # # create and connect mapper
        # cubeMapper = vtk.vtkPolyDataMapper()
        # cubeMapper.SetInputConnection(cube.GetOutputPort())
        # cubeActor.SetMapper(cubeMapper)
        # # add actor to list (no data to be kept in memory explicitely here, as cube is kept by cubeMapper and cubeMapper is kept by cubeActor)
        # actor_data_list.append((cubeActor, []))

    def add_3d_actors(self, data_package: "DataProvider", actor_data_list: list):
        """
        dummy for plot functions which can be added in derived classes, to add class specific actors in 3d view

        Args:
            data_package:       DataProvider holding data to plot
            actor_data_list:    list to add your plot data to, as tuple (vtk object actors, list with data to be kept in memory as long as data is plotted)
        """
        pass
        # do not plot anything with this function in base class, use a derived class with overloaded function
        # e.g. plot a red cube at origin:
        # import vtk
        # # create cube structure
        # cube = vtk.vtkCubeSource()
        # cube.SetCenter(0, 0, 0)
        # cube.SetXLength(4)
        # cube.SetYLength(2)
        # cube.SetZLength(1.5)
        # # create actor
        # cubeActor = vtk.vtkActor()
        # cubeActor.GetProperty().SetRepresentationToWireframe()
        # cubeActor.GetProperty().SetColor(1, 0, 0)
        # cubeActor.GetProperty().LightingOff()
        # # create and connect mapper
        # cubeMapper = vtk.vtkPolyDataMapper()
        # cubeMapper.SetInputConnection(cube.GetOutputPort())
        # cubeActor.SetMapper(cubeMapper)
        # # add actor to list (no data to be kept in memory explicitely here, as cube is kept by cubeMapper and cubeMapper is kept by cubeActor)
        # actor_data_list.append((cubeActor, []))


class DataProvider(object):
    """
    Data provider object
    """

    # static variables
    callback_dmy_cnt = 0

    # define available named sensortypes
    named_sensor_types = ["LIDAR", "RADAR", "FUSEDRADAR", "CAMERA"]

    def __init__(
        self,
        dp_meta_data: Optional[DataProviderMetaData] = None,
        tag: str = "",
        data2load: dict = {"LIDAR": [], "RADAR": [], "FUSEDRADAR": [], "CAMERA": [], "LABEL": []},
        label_converter: Optional[ClassMapper] = None,
        data_cache: Optional[dict] = None,
        version2load: dict = {},
        data_manipulators: list = [],
    ):
        """
        inits data provider object

        Args:
            dp_meta_data:       meta data structure
            tag:                optional tag for data package
            data2load:          which data schall be loaded
            label_converter:    class mapper
            data_cache:         cached data
            version2load:       optional specifications of data versions to load
            data_manipulators:  data manipulators assigned to data package
        """

        # data structures
        self.META = dict()
        self.LIDARS = dict()
        self.RADARS = dict()
        self.FUSEDRADARS = dict()
        self.CAMERAS = dict()
        self.LABEL = dict()

        # store source type
        if dp_meta_data is None:
            self.source_type = None
        else:
            self.source_type = dp_meta_data.source_type

        # tag as identifier code for data sample, create one if not provided
        self.tag = tag
        if self.tag == "":
            self.tag = "".join(random.choice(string.ascii_uppercase + string.digits) for _ in range(8))

        # read data if provided
        if dp_meta_data is not None:
            self.read_data(dp_meta_data, data2load, version2load, data_cache)

        # convert label if entered in converter
        if label_converter is not None:
            for layer in self.LABEL:
                for lab in self.LABEL[layer]:
                    if lab.breed == "annotation":
                        speed = np.linalg.norm(lab.vel_3d_filtered)
                    else:
                        speed = np.linalg.norm(lab.vel_3d)
                    mapped_type, _, _ = label_converter.map(class_in=lab.type, speed=speed)
                    if mapped_type is not None:
                        lab.type = mapped_type

        # add data handlers
        if dp_meta_data is not None:
            data_handlers = dp_meta_data.data_handlers
        else:
            data_handlers = None

        # apply data manipulators
        self.data_manipulators = []
        if len(data_manipulators) > 0:
            self.apply_data_manipulators(data_cache, data_manipulators, data2load, version2load, label_converter, data_handlers)

    def get_sensor_data(self, sensor_type: str, sensor_id: int) -> dict:
        """
        getter for data of specific sensor

        Args:
            sensor_type:    query sensor type
            sensor_id:      query sensor id

        Returns:
            query result dict
        """

        if sensor_type in self.named_sensor_types:
            out = None
            if sensor_type == "LIDAR":
                out = self.LIDARS.get(sensor_id, None)
            elif sensor_type == "RADAR":
                out = self.RADARS.get(sensor_id, None)
            elif sensor_type == "CAMERA":
                out = self.CAMERAS.get(sensor_id, None)
            elif sensor_type == "FUSEDRADAR":
                out = self.FUSEDRADARS.get(sensor_id, None)
            if out is None:
                print("WARNING: there is no sensor id %s for sensor type %s" % (sensor_id, sensor_type))
            return out
        else:
            print("WARNING: sensor type %s not supported" % sensor_type)
            return None

    def apply_data_manipulators(
        self,
        data_cache: dict,
        registered_data_manipulators: list,
        data2load: dict,
        version2load: dict,
        label_converter: ClassMapper,
        data_handlers: Optional[list],
    ):
        """
        apply data manipulators

        Args:
            data_cache:                     cached data
            registered_data_manipulators:   data manipulators assigned to data package
            data2load:                      which data schall be loaded
            version2load:                   optional specifications of data versions to load
            label_converter:                class mapper
            data_handlers:                  data handling structure
        """

        # add and perform data manipulators
        for rdm in registered_data_manipulators:
            # check type
            if not type(rdm) is tuple and len(rdm) == 2 and issubclass(rdm[0], DataManipulator) and type(rdm[1]) is dict:
                raise ValueError(
                    "Each element of data_manipulators must be tuple of (class derived from DataManipulator, argument dict) in DataProvider.__init__"
                )
            # create manipulator and perform data manipulation
            if data_cache is None:
                dc = None
            else:
                dc = data_cache[0]
            data_config = {
                "data2load": data2load,
                "version2load": version2load,
                "label_converter": label_converter,
                "data_handlers": data_handlers,
                "data_cache": dc,
            }
            dm_instance = rdm[0](self, data_config, **rdm[1])
            # add manipulator link
            self.data_manipulators.append(dm_instance)

        # perform cache independent updates as well
        self.update_data_manipulators_cache_independent(data_cache, registered_data_manipulators, data2load, version2load, label_converter, data_handlers)

    def update_data_manipulators_cache_independent(
        self,
        data_cache: dict,
        registered_data_manipulators: list,
        data2load: dict,
        version2load: dict,
        label_converter: ClassMapper,
        data_handlers: Optional[list],
    ):
        """
        update data manipulators cache independent, this functionality is called even if generator takes DataProvider from cache

        Args:
            data_cache:                     cached data
            registered_data_manipulators:   data manipulators assigned to data package
            data2load:                      which data shall be loaded
            version2load:                   optional specifications of data versions to load
            label_converter:                class mapper
            data_handlers:                  data handling structure
        """
        # get lookup of data manipulator type to manipulator input
        manipulator_input_lookup = {rdm[0]: rdm[1] for rdm in registered_data_manipulators}
        # cycle data_manipulator instances of this DataProvider and update with registered input
        for manipulator_instance in self.data_manipulators:
            # get input from lookup
            manipulator_input = manipulator_input_lookup[type(manipulator_instance)]
            # perform update
            if data_cache is None:
                dc = None
            else:
                dc = data_cache[0]
            data_config = {
                "data2load": data2load,
                "version2load": version2load,
                "label_converter": label_converter,
                "data_handlers": data_handlers,
                "data_cache": dc,
            }
            manipulator_instance.update_cache_independent(self, data_config, **manipulator_input)

    @staticmethod
    def callback_dmy() -> bool:
        """
        dummy for a callback function that could be connected to a viewer with
        Viewer.register_callback_function('dmy_name', DataProvider.callback_dmy)

        Returns:
            requires repaint
        """
        print("callback_dmy_cnt=%d" % DataProvider.callback_dmy_cnt)
        print("Increase callback_dmy_cnt")
        DataProvider.callback_dmy_cnt += 1
        # return True if repaint is required after callback
        return False

    def get_byte_size(self) -> int:
        """
        get approximate size of DataProvider object in byte

        Returns:
            approximate size of DataProvider object in byte
        """
        sz = 0
        for val in self.LIDARS.values():
            sz += sys.getsizeof(val.pointcloud)
        for val in self.RADARS.values():
            sz += sys.getsizeof(val.pointcloud)
        for val in self.FUSEDRADARS.values():
            sz += sys.getsizeof(val.pointcloud)
        for val in self.CAMERAS.values():
            sz += sys.getsizeof(val.image)
        return sz

    def read_data(self, dp_meta_data: DataProviderMetaData, data2load: dict, version2load: dict, data_cache: dict):
        """
        read data and fill it into DataProvider structures

        Args:
            dp_meta_data:   meta data structure
            data2load:      which data schall be loaded
            version2load:   optional specifications of data versions to load
            data_cache:     cached data
        """
        # read data
        data = DataProvider.read(dp_meta_data, data2load, version2load, data_cache)
        # create data package
        self.META = data["META"]
        self.LIDARS = data["LIDARS"]
        self.RADARS = data["RADARS"]
        self.FUSEDRADARS = data["FUSEDRADARS"]
        self.CAMERAS = data["CAMERAS"]
        self.LABEL = data["LABEL"]
        # create sensor name to index lookups
        self.META["sensor_name_to_idx_lookup"] = dict()
        for stype in DataProvider.named_sensor_types:
            self.META["sensor_name_to_idx_lookup"][stype] = dict()
            for sid, sdat in data[stype + "S"].items():
                self.META["sensor_name_to_idx_lookup"][stype][sdat.sensor_name] = sid

    @staticmethod
    def read(dp_meta_data: DataProviderMetaData, data2load: dict, version2load: dict, data_cache: dict) -> dict:
        """
        the read function which is called by the DataProvider

        Args:
            dp_meta_data:   meta data structure
            data2load:      which data schall be loaded
            version2load:   optional specifications of data versions to load
            data_cache:     cached data

        Returns:
            data dictionary
        """
        source_typs = {"trainingTool2": DataProvider.read_training_tooling_2, "cbor": DataProvider.read_cbor}

        load_function = source_typs.get(dp_meta_data.source_type, lambda dp_meta_data, data2load: print("Unknown load mode: " + dp_meta_data.source_type))
        return load_function(dp_meta_data, data2load, version2load, data_cache)

    @staticmethod
    def read_training_tooling_2(dp_meta_data: DataProviderMetaData, data2load: dict, version2load: dict = {}, data_cache: Optional[dict] = None) -> dict:
        """
        The training_tooling_2 read function:
        Every read function should return the data in the structure the DataProvider expects. This means a dictionary with
        the following members:
          LIDARS: Dict of lidar objects, each containing a pointcloud and a lidar calibration object
          RADARS: Dict of radar objects, each containing a pointcloud and a radar calibration object
          FUSEDRADARS: Dict of virtual radar sensors combining the data of a list of real sensors
          CAMERAS: Dict of camera objects, each containing a pointcloud and a camera calibration object
          LABEL: Dict of Label layers, each containing a label list

        Args:
            dp_meta_data:   meta data structure
            data2load:      which data schall be loaded
            version2load:   optional specifications of data versions to load
            data_cache:     cached data

        Returns:
            data dictionary
        """

        # filenames
        meta_file_name = os.path.join(dp_meta_data.folder, "Meta", "%s.h5" % dp_meta_data.sequence_name)
        lidar_file_name = os.path.join(dp_meta_data.folder, "Lidar", "%s.h5" % dp_meta_data.sequence_name)
        camera_file_name = os.path.join(dp_meta_data.folder, "Camera", "%s.h5" % dp_meta_data.sequence_name)
        radar_detections_file_name = os.path.join(dp_meta_data.folder, "Radar_Detections", "%s.h5" % dp_meta_data.sequence_name)

        # check if data cache contains relevant information
        data_cache_relevant = (
            data_cache is not None and data_cache[1].folder == dp_meta_data.folder and data_cache[1].sequence_name == dp_meta_data.sequence_name
        )

        # load radar
        radars = dict()
        if "RADAR" in data2load.keys() and os.path.exists(radar_detections_file_name):
            radar_file = h5py.File(radar_detections_file_name, "r")
            meta_file = h5py.File(meta_file_name, "r")
            for rad_id in dp_meta_data.aligned_fids["RADAR"]:
                if dp_meta_data.aligned_fids["RADAR"][rad_id] is not None:
                    if data_cache_relevant and data_cache[1].aligned_fids["RADAR"][rad_id] == dp_meta_data.aligned_fids["RADAR"][rad_id]:
                        radars[rad_id] = data_cache[0].RADARS[rad_id]
                    else:
                        rad_name = radar_file["radar_idx_to_name"][rad_id]
                        rad_gp = radar_file["sensors"][rad_name]
                        # get current frame
                        frame = dp_meta_data.aligned_fids["RADAR"][rad_id]
                        ## get pointcloud
                        radar_version = version2load.get("RADAR", "v0")  # deactivate pointcloud if version is None
                        # get slice
                        slice = get_slice(rad_gp, frame)
                        if radar_version is not None and slice[1] - slice[0] > 0:
                            # get pointcloud
                            radar_pc = np.zeros([16, slice[1] - slice[0]], dtype=np.float32)
                            # get data
                            cols = rad_gp["data"].attrs["columns"].split(";")
                            lookup = dict(zip(cols, range(len(cols))))
                            cols2slice = [lookup[kk] for kk in ["x", "y", "z", "rcs", "rangeRate"]]
                            radar_pc[0:5, :] = rad_gp["data"][slice[0] : slice[1], :][:, cols2slice].T
                            # radar_pc[5:10, :] = ['doppl_propasal_0', 'doppl_propasal_1', 'doppl_propasal_2', 'doppl_propasal_3', 'doppl_propasal_4']
                            radar_pc[10, :] = rad_gp["looktype"][frame, 0]
                            # radar_pc[11, :] ['isSingleTarget']
                            radar_pc[12, :] = rad_id
                            radar_pc[13, :] = rad_gp["timestamps"][frame, 0]
                            radar_pc[14, :] = frame
                            radar_pc[15, :] = rad_gp["data"][slice[0]:slice[1], -1].T
                        else:
                            radar_pc = None
                        # create radar calib
                        radar_calib = RadarCalib()
                        radar_calib.T_V2R = rad_gp["calib"]["extrinsic"][()].T
                        radar_calib.T_R2V = np.linalg.inv(radar_calib.T_V2R)
                        radar_calib.aperture_angle = meta_file["radar_aperture_angle"][rad_id, :]
                        # create radar and fill meta information
                        radars[rad_id] = Radar(
                            pointcloud=radar_pc,
                            radar_calib=radar_calib,
                            timestamp=dp_meta_data.aligned_timestamps["RADAR"][rad_id],
                            frame_id=dp_meta_data.aligned_fids["RADAR"][rad_id],
                            scan_id=dp_meta_data.aligned_sids["RADAR"][rad_id],
                            sensor_name=dp_meta_data.aligned_sensor_names["RADAR"][rad_id],
                            version=None,
                            subsensors=None,
                        )
                        radars[rad_id].is_fused = False

                        # load egomotion file
                        ego_gp = rad_gp["ego_motion"]
                        radars[rad_id].ego_transform = ego_gp["T0"][frame].T
                        radars[rad_id].ego_transform_to_previous = ego_gp["T"][frame].T
                        radars[rad_id].v_lon, radars[rad_id].v_lat, radars[rad_id].yawrate = ego_gp["ego_scalars"][frame]

            # close file handle
            radar_file.close()
            meta_file.close()

        # load fused radar
        fusedradars = dict()
        if "FUSEDRADAR" in data2load.keys():
            if dp_meta_data.aligned_fids["FUSEDRADAR"][0] is not None:
                if data_cache_relevant and data_cache[1].aligned_fids["FUSEDRADAR"][0] == dp_meta_data.aligned_fids["FUSEDRADAR"][0]:
                    fusedradars[0] = data_cache[0].FUSEDRADARS[0]
                else:
                    if os.path.exists(meta_file_name):
                        meta_file = h5py.File(meta_file_name, "r")
                        frame = dp_meta_data.aligned_fids["FUSEDRADAR"][0]
                        ## get pointcloud and calib
                        radar_pc = None
                        radar_version = version2load.get("FUSEDRADAR", "v0")  # deactivate pointcloud if version is None
                        radar_calib = dict()
                        T_V2R = meta_file["radar_calib"][()].transpose(0, 2, 1)  # TODO: Check me!!!
                        for rad_id, rad in dp_meta_data.aligned_subsensors["FUSEDRADAR"][0].items():
                            radar_calib[rad_id] = RadarCalib()
                            radar_calib[rad_id].T_V2R = T_V2R[rad_id]
                            radar_calib[rad_id].T_R2V = np.linalg.inv(T_V2R[rad_id])
                            radar_calib[rad_id].aperture_angle = meta_file["radar_aperture_angle"][rad_id, :]
                        if os.path.exists(radar_detections_file_name):
                            radar_file = h5py.File(radar_detections_file_name, "r")
                            # load batch to radar frame mapping
                            batch2radar_map = radar_file["batch_to_radar_frame_map"]
                            # fill local pointcloud
                            pointclouds = []
                            for rad_id, rad in dp_meta_data.aligned_subsensors["FUSEDRADAR"][0].items():
                                if rad is not None:
                                    rad_gp = radar_file["sensors"][rad["sensor_name"]]
                                    # get slice
                                    radar_frm = batch2radar_map[frame, rad_id]
                                    if radar_frm >= 0:
                                        slice = get_slice(rad_gp, radar_frm)
                                        if radar_version is not None and slice[1] - slice[0] > 0:
                                            # get pointcloud
                                            radar_pc_loc = np.zeros([16, slice[1] - slice[0]], dtype=np.float32)
                                            # get data
                                            cols = rad_gp["data"].attrs["columns"].split(";")
                                            lookup = dict(zip(cols, range(len(cols))))
                                            cols2slice = [lookup[kk] for kk in ["x", "y", "z", "rcs", "rangeRate"]]
                                            radar_pc_loc[0:5, :] = rad_gp["data"][slice[0] : slice[1], :][:, cols2slice].T
                                            # radar_pc[5:10, :] = ['doppl_propasal_0', 'doppl_propasal_1', 'doppl_propasal_2', 'doppl_propasal_3', 'doppl_propasal_4']
                                            radar_pc_loc[10, :] = rad_gp["looktype"][radar_frm, 0]
                                            # radar_pc[11, :] ['isSingleTarget']
                                            radar_pc_loc[12, :] = rad_id
                                            radar_pc_loc[13, :] = rad_gp["timestamps"][radar_frm, 0]
                                            radar_pc_loc[14, :] = radar_frm
                                            radar_pc_loc[15, :] = rad_gp["data"][slice[0]:slice[1], -1].T
                                            pointclouds.append(radar_pc_loc)
                            # concatenate pointclouds
                            if len(pointclouds) > 0:
                                radar_pc = np.hstack(pointclouds)
                            # close file handle
                            radar_file.close()
                        # create radar and fill meta information
                        fusedradars[0] = Radar(
                            pointcloud=radar_pc,
                            radar_calib=radar_calib,
                            timestamp=dp_meta_data.aligned_timestamps["FUSEDRADAR"][0],
                            frame_id=dp_meta_data.aligned_fids["FUSEDRADAR"][0],
                            scan_id=None,
                            sensor_name=dp_meta_data.aligned_sensor_names["FUSEDRADAR"][0],
                            version=None,
                            subsensors=dp_meta_data.aligned_subsensors["FUSEDRADAR"][0],
                        )
                        fusedradars[0].is_fused = True

                        # load egomotion file
                        fusedradars[0].ego_transform = meta_file["T0"][frame].T
                        fusedradars[0].ego_transform_to_previous = meta_file["T"][frame].T
                        fusedradars[0].v_lon, fusedradars[0].v_lat, fusedradars[0].yawrate = meta_file["ego_scalars"][frame]

                        # close file handle
                        meta_file.close()

        # load lidar
        lidars = dict()
        if "LIDAR" in data2load.keys() and os.path.exists(lidar_file_name):
            lidar_file = h5py.File(lidar_file_name, "r")
            for lid_id in dp_meta_data.aligned_fids["LIDAR"]:
                if dp_meta_data.aligned_fids["LIDAR"][lid_id] is not None:
                    if data_cache_relevant and data_cache[1].aligned_fids["LIDAR"][lid_id] == dp_meta_data.aligned_fids["LIDAR"][lid_id]:
                        lidars[lid_id] = data_cache[0].LIDARS[lid_id]
                    else:
                        lid_name = lidar_file["lidar_idx_to_name"][lid_id]
                        lid_gp = lidar_file["sensors"][lid_name]
                        # get current frame
                        frame = dp_meta_data.aligned_fids["LIDAR"][lid_id]
                        # slice
                        slice = get_slice(lid_gp, frame)
                        # reshape
                        lidar_pc = np.zeros([9, slice[1] - slice[0]], dtype=np.float64)
                        lidar_pc[0:4, :] = lid_gp["data"][slice[0] : slice[1], :].T
                        # normalize
                        lidar_pc[3, :] /= 255
                        # set timestamp
                        lidar_pc[4, :] = (lid_gp["timestamps"][frame] + lid_gp["data_auxiliary"][slice[0] : slice[1], 0]) * 1000
                        # set file id
                        lidar_pc[5, :] = frame
                        # prepare label, instance_id, stationary_flag
                        lidar_pc[6:9, :] = -1
                        # create calib
                        lidar_calib = LidarCalib()
                        lidar_calib.T_V2L = lid_gp["calib"]["extrinsic"][()].T
                        lidar_calib.T_L2V = np.linalg.inv(lidar_calib.T_V2L)
                        lidars[lid_id] = Lidar(
                            pointcloud=lidar_pc,
                            lidar_calib=lidar_calib,
                            timestamp=dp_meta_data.aligned_timestamps["LIDAR"][lid_id],
                            frame_id=dp_meta_data.aligned_fids["LIDAR"][lid_id],
                            scan_id=dp_meta_data.aligned_sids["LIDAR"][lid_id],
                            sensor_name=dp_meta_data.aligned_sensor_names["LIDAR"][lid_id],
                        )

                        # load egomotion
                        ego_gr = lid_gp["ego_motion"]
                        lidars[lid_id].ego_transform = ego_gr["T0"][frame].T
                        lidars[lid_id].ego_transform_to_previous = ego_gr["T"][frame].T
                        lidars[lid_id].v_lon, lidars[lid_id].v_lat, lidars[lid_id].yawrate = ego_gr["ego_scalars"][frame]

            # close handle
            lidar_file.close()

        # load labels
        labels = {"GT": []}
        label_GT_src = None
        if "LABEL" in data2load.keys():
            if data_cache_relevant and data_cache[1].aligned_fids["LABEL"] == dp_meta_data.aligned_fids["LABEL"]:
                labels = data_cache[0].LABEL
            else:
                if dp_meta_data.aligned_fids["LABEL"] is not None:
                    labels_filename = dp_meta_data.aligned_fids["LABEL"]["file"]
                    if os.path.exists(labels_filename):
                        label_file = h5py.File(labels_filename, "r")
                        # create h5 handles for all sensors to load
                        if "sensors" in label_file:
                            # for Lidar and single radars we use sensorwise label file
                            label_sensor_gp = label_file["sensors"][dp_meta_data.aligned_fids["LABEL"]["sensor"]]
                        else:
                            # for everything else use the FUSEDRADAR label group
                            label_sensor_gp = label_file
                        # get unique list combining ground truth source from dataspec and additional sources from data2load
                        GT_src = dp_meta_data.aligned_fids["LABEL"]["source"]
                        sources2load = list(set([GT_src] + data2load["LABEL"]))
                        # cycle sources
                        for lab_src in sources2load:
                            # skip label source None
                            if lab_src is None:
                                continue
                            labels[lab_src] = []
                            if lab_src.encode("utf-8") in label_sensor_gp["label_idx_to_name"]:
                                label_gp = label_sensor_gp["label_sources"][lab_src]
                                frame_range = label_gp["frame_range"]
                                # capture trivial case were no labels are available at all
                                if label_gp["data"].shape[0] == 0:
                                    continue
                                data_UUID_mapping = [uuid.decode("utf-8") for uuid in label_gp["data_UUID_mapping"]]
                                # get frame
                                frame = dp_meta_data.aligned_fids["LABEL"]["frame"]
                                if frame_range[0] <= frame <= frame_range[1]:
                                    # slice
                                    slice = get_slice(label_gp, frame)
                                    data = label_gp["data"][slice[0] : slice[1], :]
                                    cols = label_gp["data"].attrs["columns"].split(";")
                                    lookup = dict(zip(cols, range(len(cols))))
                                    for i in range(data.shape[0]):
                                        label = Label()
                                        # class
                                        if "labels_mapping" in label_gp:
                                            label.type = label_gp["labels_mapping"][data[i][lookup["Label"]].astype(int)].decode("utf-8")
                                        else:
                                            label.type = label_gp["data_Label_mapping"][data[i][lookup["Label"]].astype(int)].decode("utf-8")
                                        # position
                                        label.loc_3d[0] = data[i][lookup["XPos"]]
                                        label.loc_3d[1] = data[i][lookup["YPos"]]
                                        label.loc_3d[2] = data[i][lookup["ZPos"]]
                                        # dimension (length, width, height)
                                        label.dim_3d[0] = data[i][lookup["XDim"]]
                                        label.dim_3d[1] = data[i][lookup["YDim"]]
                                        label.dim_3d[2] = data[i][lookup["ZDim"]]
                                        # rotation as 3d matrix vectorized along columns
                                        # cx = math.cos(data['Roll'][i])
                                        # sx = math.sin(data['Roll'][i])
                                        # cy = math.cos(data['Pitch'][i])
                                        # sy = math.sin(data['Pitch'][i])
                                        yaw = data[i][lookup["Yaw"]]
                                        cz = math.cos(yaw)
                                        sz = math.sin(yaw)
                                        # label.rot_3d[0, 0] = cy * cz
                                        # label.rot_3d[0, 1] = sx * sy * cz - cx * sz
                                        # label.rot_3d[0, 2] = cx * sy * cz + sx * sz
                                        # label.rot_3d[1, 0] = cy * sz
                                        # label.rot_3d[1, 1] = sx * sy * sz + cx * cz
                                        # label.rot_3d[1, 2] = cx * sy * sz - sx * cz
                                        # label.rot_3d[2, 0] = -sy
                                        # label.rot_3d[2, 1] = sx * cy
                                        # label.rot_3d[2, 2] = cx * cy
                                        label.rot_3d[0, 0] = cz
                                        label.rot_3d[0, 1] = -sz
                                        label.rot_3d[0, 2] = 0.0
                                        label.rot_3d[1, 0] = sz
                                        label.rot_3d[1, 1] = cz
                                        label.rot_3d[1, 2] = 0.0
                                        label.rot_3d[2, 0] = 0.0
                                        label.rot_3d[2, 1] = 0.0
                                        label.rot_3d[2, 2] = 1.0
                                        # velocity and acceleration
                                        label.vel_3d_filtered[0] = data[i][lookup["XVelFiltered"]]
                                        label.vel_3d_filtered[1] = data[i][lookup["YVelFiltered"]]
                                        label.vel_3d_filtered[2] = 0
                                        label.acc_3d_filtered[0] = data[i][lookup["XAccFiltered"]]
                                        label.acc_3d_filtered[1] = data[i][lookup["YAccFiltered"]]
                                        label.acc_3d_filtered[2] = 0
                                        if "XVel" in lookup:
                                            label.vel_3d[0] = data[i][lookup["XVel"]]
                                            label.vel_3d[1] = data[i][lookup["YVel"]]
                                            label.vel_3d[2] = data[i][lookup["ZVel"]]
                                        else:
                                            label.vel_3d = label.vel_3d_filtered.copy()
                                        if "XAcc" in lookup:
                                            label.acc_3d[0] = data[i][lookup["XAcc"]]
                                            label.acc_3d[1] = data[i][lookup["YAcc"]]
                                            label.acc_3d[2] = data[i][lookup["ZAcc"]]
                                        else:
                                            label.acc_3d = label.acc_3d_filtered.copy()
                                        # track id
                                        label.track_id = data_UUID_mapping[int(data[i][lookup["UUID"]])]  # int(data[i][lookup['UUID']])
                                        # no care flag
                                        if "NoCare" in lookup:
                                            label.no_care = data[i][lookup["NoCare"]]
                                        else:
                                            label.no_care = False
                                        if "geom_NoCare" in lookup:
                                            label.geom_no_care = data[i][lookup["geom_NoCare"]]
                                        else:
                                            label.geom_no_care = False
                                        # visibility
                                        vis_entries = [vv for vv in lookup if "Visibility" in vv]
                                        if len(vis_entries) > 0:
                                            label.visibility = {int(ve.split("_")[1]): data[i][lookup[ve]] for ve in vis_entries}
                                        # mark as annotation
                                        label.breed = "annotation"

                                        labels[lab_src].append(label)
                        # rename round truth source to 'GT'
                        if GT_src in labels:
                            label_GT_src = GT_src
                            labels["GT"] = labels.pop(GT_src)
                        # close file handle
                        label_file.close()

        # load cameras
        cameras = dict()
        if "CAMERA" in data2load.keys() and os.path.exists(camera_file_name):
            camera_file = h5py.File(camera_file_name, "r")
            for cam_id in dp_meta_data.aligned_fids["CAMERA"]:
                if dp_meta_data.aligned_fids["CAMERA"][cam_id] is not None:
                    if data2load["CAMERA"] == [] or cam_id in data2load["CAMERA"]:
                        if data_cache_relevant and data_cache[1].aligned_fids["CAMERA"][cam_id] == dp_meta_data.aligned_fids["CAMERA"][cam_id]:
                            cameras[cam_id] = data_cache[0].CAMERAS[cam_id]
                        else:
                            # get handle
                            cam_name = camera_file["camera_idx_to_name"][cam_id]
                            cam_gp = camera_file["sensors"][cam_name]
                            # get current frame
                            frame = dp_meta_data.aligned_fids["CAMERA"][cam_id]
                            # get image
                            img = cv2.imdecode(cam_gp["data"][frame], cv2.IMREAD_COLOR)
                            # get calib
                            # read inner calibration
                            K = cam_gp["calib"]["intrinsic"][()].T
                            P = np.matmul(K, np.eye(3, 4))

                            # combine with L2V to get camera to vehicle
                            T_V2C = cam_gp["calib"]["extrinsic"][()].T

                            ####### some code to change/test camera calibration #######
                            # from scipy.spatial.transform import Rotation as Rot
                            # correction_angles_degree = {
                            #     'ITC_GoFast_Ladybug_Front': [0, 0, -5],
                            #     'ITC_GoFast_Ladybug_Rear_Left': [0, 0, 0],
                            #     'ITC_GoFast_Ladybug_Front_Left': [0, 0, 0],
                            #     'ITC_GoFast_Ladybug_Rear_Right': [0, 0, 0],
                            #     'ITC_GoFast_Ladybug_Front_Right': [0, 0, 0],
                            # }
                            # T_C2V = np.linalg.inv(T_V2C)
                            # R_correction = Rot.from_euler('xyz', correction_angles_degree[cam_name.decode('utf-8')], degrees=True).as_matrix()
                            # R_axis = np.array([[0, 0, 1],
                            #                    [1, 0, 0],
                            #                    [0, 1, 0]])
                            # R_correction = R_correction @ R_axis
                            #
                            # T_C2V[:3, :3] = T_C2V[:3, :3] @ R_correction
                            # T_V2C = np.linalg.inv(T_C2V)
                            # print(cam_name.decode('utf-8'), Rot.from_matrix(T_C2V[:3, :3]).as_quat())

                            # create camera structure
                            cameracalib = CameraCalib(
                                data={"img_width": cam_gp["calib"]["img_size"][0], "img_height": cam_gp["calib"]["img_size"][1], "T_V2C": T_V2C, "P": P}
                            )
                            # create camera object
                            cameras[cam_id] = Camera(
                                image=img,
                                camera_calib=cameracalib,
                                timestamp=dp_meta_data.aligned_timestamps["CAMERA"][cam_id],
                                frame_id=dp_meta_data.aligned_fids["CAMERA"][cam_id],
                                scan_id=dp_meta_data.aligned_sids["CAMERA"][cam_id],
                                sensor_name=dp_meta_data.aligned_sensor_names["CAMERA"][cam_id],
                            )

                            # load egomotion
                            ego_gr = cam_gp["ego_motion"]
                            cameras[cam_id].ego_transform = ego_gr["T0"][frame].T
                            cameras[cam_id].ego_transform_to_previous = ego_gr["T"][frame].T
                            cameras[cam_id].v_lon, cameras[cam_id].v_lat, cameras[cam_id].yawrate = ego_gr["ego_scalars"][frame]

            # close handle
            camera_file.close()

        # get meta information
        data_info = None
        vehicle_info = None
        if os.path.exists(meta_file_name):
            meta_file = h5py.File(meta_file_name, "r")
            data_info = dict(meta_file["info"].attrs)
            vehicle_info = dict(meta_file["vehicle_info"].attrs)
            meta_file.close()

        # create result structure
        ret = {
            "META": {
                "sensor_type": dp_meta_data.sensor_type,
                "sensor_id": dp_meta_data.sensor_id,
                "sensor_name": dp_meta_data.sensor_name,
                "sensor_frame_id": dp_meta_data.aligned_fids[dp_meta_data.sensor_type][dp_meta_data.sensor_id],
                "folder": dp_meta_data.folder,
                "sequence_name": dp_meta_data.sequence_name,
                "timestamp": dp_meta_data.aligned_timestamps[dp_meta_data.sensor_type][dp_meta_data.sensor_id],
                "GT_source": label_GT_src,
                "data_info": data_info,
                "vehicle_info": vehicle_info,
            },
            "LIDARS": lidars,
            "RADARS": radars,
            "FUSEDRADARS": fusedradars,
            "CAMERAS": cameras,
            "LABEL": labels,
        }
        return ret

    @staticmethod
    def read_cbor(dp_meta_data: DataProviderMetaData, data2load: dict, version2load: dict = {}, data_cache: Optional[dict] = None) -> dict:
        """
        The cbor read function:
        Every read function should return the data in the structure the DataProvider expects. This means a dictionary with
        the following members:
          LIDARS: Dict of lidar objects, each containing a pointcloud and a lidar calibration object
          RADARS: Dict of radar objects, each containing a pointcloud and a radar calibration object
          FUSEDRADARS: Dict of virtual radar sensors combining the data of a list of real sensors
          CAMERAS: Dict of camera objects, each containing a pointcloud and a camera calibration object
          LABEL: Dict of Label layers, each containing a label list

        Args:
            dp_meta_data:   meta data structure
            data2load:      which data schall be loaded
            version2load:   optional specifications of data versions to load
            data_cache:     cached data

        Returns:
            data dictionary
        """

        # check if data cache contains relevant information
        data_cache_relevant = (
            data_cache is not None and data_cache[1].folder == dp_meta_data.folder and data_cache[1].sequence_name == dp_meta_data.sequence_name
        )

        # get handle to cbor handler
        cbor_handler = dp_meta_data.data_handlers["cbor_handler"]

        # load radar
        radars = dict()
        radar_data = cbor_handler.get_data().get("radar", None)
        if "RADAR" in data2load.keys() and radar_data:
            for rad_id, rad_name in dp_meta_data.aligned_sensor_names["RADAR"].items():
                if dp_meta_data.aligned_fids["RADAR"][rad_id] is not None:
                    if data_cache_relevant and data_cache[1].aligned_fids["RADAR"][rad_id] == dp_meta_data.aligned_fids["RADAR"][rad_id]:
                        radars[rad_id] = data_cache[0].RADARS[rad_id]
                    else:
                        rad_gp = radar_data[rad_name]
                        # get current frame
                        frame = dp_meta_data.aligned_fids["RADAR"][rad_id]
                        ## get pointcloud
                        radar_version = version2load.get("RADAR", "v0")  # deactivate pointcloud if version is None
                        if radar_version is not None and rad_gp["data"][frame].shape[1] > 0:
                            # get pointcloud
                            radar_pc = np.zeros([15, rad_gp["data"][frame].shape[1]], dtype=np.float32)
                            # get data
                            radar_pc[:5, :] = rad_gp["data"][frame][:5, :]  # ['x', 'y', 'z', 'rcs', 'rangeRate']
                            # radar_pc[5:10, :] = ['doppl_propasal_0', 'doppl_propasal_1', 'doppl_propasal_2', 'doppl_propasal_3', 'doppl_propasal_4']
                            radar_pc[10, :] = rad_gp["looktype"][frame]
                            # radar_pc[11, :] ['isSingleTarget']
                            radar_pc[12, :] = rad_id
                            radar_pc[13, :] = rad_gp["timestamps"][frame]
                            radar_pc[14, :] = frame
                        else:
                            radar_pc = None
                        # create radar calib
                        radar_calib = RadarCalib()
                        radar_calib.T_V2R = rad_gp["calib"]["T_V2R"]
                        radar_calib.T_R2V = np.linalg.inv(radar_calib.T_V2R)
                        radar_calib.aperture_angle = rad_gp["calib"]["aperture_angle"]
                        # create radar and fill meta information
                        radars[rad_id] = Radar(
                            pointcloud=radar_pc,
                            radar_calib=radar_calib,
                            timestamp=dp_meta_data.aligned_timestamps["RADAR"][rad_id],
                            frame_id=dp_meta_data.aligned_fids["RADAR"][rad_id],
                            scan_id=dp_meta_data.aligned_sids["RADAR"][rad_id],
                            sensor_name=dp_meta_data.aligned_sensor_names["RADAR"][rad_id],
                            version=None,
                            subsensors=None,
                        )
                        radars[rad_id].is_fused = False

                        # load egomotion
                        ego_gp = rad_gp["ego_motion"]
                        radars[rad_id].ego_transform = ego_gp["T0"][frame]
                        radars[rad_id].ego_transform_to_previous = ego_gp["T"][frame]
                        radars[rad_id].v_lon, radars[rad_id].v_lat, radars[rad_id].yawrate = ego_gp["ego_scalars"][frame]

        # load fused radar
        fusedradars = dict()
        radar_data = cbor_handler.get_data().get("radar", None)
        if "FUSEDRADAR" in data2load.keys() and radar_data:
            if dp_meta_data.aligned_fids["FUSEDRADAR"][0] is not None:
                if data_cache_relevant and data_cache[1].aligned_fids["FUSEDRADAR"][0] == dp_meta_data.aligned_fids["FUSEDRADAR"][0]:
                    fusedradars[0] = data_cache[0].FUSEDRADARS[0]
                else:
                    frame = dp_meta_data.aligned_fids["FUSEDRADAR"][0]
                    ## get pointcloud and calib
                    radar_pc = None
                    radar_version = version2load.get("FUSEDRADAR", "v0")  # deactivate pointcloud if version is None
                    radar_calib = dict()

                    # fill local pointcloud
                    pointclouds = []
                    for rad_id, rad in dp_meta_data.aligned_subsensors["FUSEDRADAR"][0].items():
                        if rad is not None:
                            rad_gp = radar_data[rad["sensor_name"]]
                            # get slice
                            radar_frm = rad["frame_id"]
                            if radar_frm >= 0:
                                if radar_version is not None:
                                    # get pointcloud
                                    radar_pc_loc = np.zeros([15, rad_gp["data"][radar_frm].shape[1]], dtype=np.float32)
                                    # get data
                                    radar_pc_loc[:5, :] = rad_gp["data"][radar_frm][:5, :]  # ['x', 'y', 'z', 'rcs', 'rangeRate']
                                    # radar_pc[5:10, :] = ['doppl_propasal_0', 'doppl_propasal_1', 'doppl_propasal_2', 'doppl_propasal_3', 'doppl_propasal_4']
                                    radar_pc_loc[10, :] = rad_gp["looktype"][radar_frm]
                                    # radar_pc[11, :] ['isSingleTarget']
                                    radar_pc_loc[12, :] = rad_id
                                    radar_pc_loc[13, :] = rad_gp["timestamps"][radar_frm]
                                    radar_pc_loc[14, :] = radar_frm
                                    pointclouds.append(radar_pc_loc)
                            # fill calib
                            T_V2R = rad_gp["calib"]["T_V2R"]
                            radar_calib[rad_id] = RadarCalib()
                            radar_calib[rad_id].T_V2R = T_V2R
                            radar_calib[rad_id].T_R2V = np.linalg.inv(T_V2R)
                            radar_calib[rad_id].aperture_angle = rad_gp["calib"]["aperture_angle"]
                    # concatenate pointclouds
                    if len(pointclouds) > 0:
                        radar_pc = np.hstack(pointclouds)

                    # create radar and fill meta information
                    fusedradars[0] = Radar(
                        pointcloud=radar_pc,
                        radar_calib=radar_calib,
                        timestamp=dp_meta_data.aligned_timestamps["FUSEDRADAR"][0],
                        frame_id=frame,
                        scan_id=None,
                        sensor_name=dp_meta_data.aligned_sensor_names["FUSEDRADAR"][0],
                        version=None,
                        subsensors=dp_meta_data.aligned_subsensors["FUSEDRADAR"][0],
                    )
                    fusedradars[0].is_fused = True

                    # load egomotion
                    ego_gr = cbor_handler.get_data()["network_input"]["network"]["ego_motion"]
                    fusedradars[0].ego_transform = ego_gr["T0"][frame]
                    fusedradars[0].ego_transform_to_previous = ego_gr["T"][frame]
                    fusedradars[0].v_lon, fusedradars[0].v_lat, fusedradars[0].yawrate = ego_gr["ego_scalars"][frame]

        # load lidar
        lidars = dict()
        lidar_data = cbor_handler.get_data().get("lidar", None)
        if "LIDAR" in data2load.keys() and lidar_data:
            for lid_id, lid_name in dp_meta_data.aligned_sensor_names["LIDAR"].items():
                if dp_meta_data.aligned_fids["LIDAR"][lid_id] is not None:
                    if data_cache_relevant and data_cache[1].aligned_fids["LIDAR"][lid_id] == dp_meta_data.aligned_fids["LIDAR"][lid_id]:
                        lidars[lid_id] = data_cache[0].LIDARS[lid_id]
                    else:
                        lid_gp = lidar_data[lid_name]
                        # get current frame
                        frame = dp_meta_data.aligned_fids["LIDAR"][lid_id]
                        # get data
                        pc = lid_gp["data"][frame]
                        # reshape
                        lidar_pc = np.zeros([9, pc.shape[1]], dtype=np.float64)
                        lidar_pc[0:4, :] = pc[:4, :]
                        # normalize
                        lidar_pc[3, :] /= 255
                        # set timestamp
                        if lidar_pc.size > 0:
                            lidar_pc[4, :] = (lid_gp["timestamps"][frame] + (pc[4, :] - np.min(pc[4, :]))) * 1000
                        # set file id
                        lidar_pc[5, :] = frame
                        # prepare label, instance_id, stationary_flag
                        lidar_pc[6:9, :] = -1
                        # create calib
                        lidar_calib = LidarCalib()
                        lidar_calib.T_V2L = lid_gp["calib"]["T_V2L"]
                        lidar_calib.T_L2V = lid_gp["calib"]["T_L2V"]
                        lidars[lid_id] = Lidar(
                            pointcloud=lidar_pc,
                            lidar_calib=lidar_calib,
                            timestamp=dp_meta_data.aligned_timestamps["LIDAR"][lid_id],
                            frame_id=dp_meta_data.aligned_fids["LIDAR"][lid_id],
                            scan_id=dp_meta_data.aligned_sids["LIDAR"][lid_id],
                            sensor_name=dp_meta_data.aligned_sensor_names["LIDAR"][lid_id],
                        )

                        # load egomotion
                        ego_gr = lid_gp["ego_motion"]
                        lidars[lid_id].ego_transform = ego_gr["T0"][frame]
                        lidars[lid_id].ego_transform_to_previous = ego_gr["T"][frame]
                        lidars[lid_id].v_lon, lidars[lid_id].v_lat, lidars[lid_id].yawrate = ego_gr["ego_scalars"][frame]

        # load labels
        labels = {"GT": []}
        label_GT_src = None
        # if 'LABEL' in data2load.keys():
        #     if data_cache_relevant and data_cache[1].aligned_fids['LABEL'] == dp_meta_data.aligned_fids['LABEL']:
        #         labels = data_cache[0].LABEL
        #     else:
        #         if dp_meta_data.aligned_fids['LABEL'] is not None:
        #             labels_filename = dp_meta_data.aligned_fids['LABEL']['file']
        #             if os.path.exists(labels_filename):
        #                 label_file = h5py.File(labels_filename, 'r')
        #                 # create h5 handles for all sensors to load
        #                 if 'sensors' in label_file:
        #                     # for Lidar and single radars we use sensorwise label file
        #                     label_sensor_gp = label_file['sensors'][dp_meta_data.aligned_fids['LABEL']['sensor']]
        #                 else:
        #                     # for everything else use the FUSEDRADAR label group
        #                     label_sensor_gp = label_file
        #                 # get unique list combining ground truth source from dataspec and additional sources from data2load
        #                 GT_src = dp_meta_data.aligned_fids['LABEL']['source']
        #                 sources2load = list(set([GT_src] + data2load['LABEL']))
        #                 # cycle sources
        #                 for lab_src in sources2load:
        #                     # skip label source None
        #                     if lab_src is None:
        #                         continue
        #                     labels[lab_src] = []
        #                     if lab_src.encode('utf-8') in label_sensor_gp['label_idx_to_name']:
        #                         label_gp = label_sensor_gp['label_sources'][lab_src]
        #                         frame_range = label_gp['frame_range']
        #                         # capture trivial case were no labels are available at all
        #                         if label_gp['data'].shape[0] == 0:
        #                             continue
        #                         data_UUID_mapping = [uuid.decode('utf-8') for uuid in label_gp['data_UUID_mapping']]
        #                         # get frame
        #                         frame = dp_meta_data.aligned_fids['LABEL']['frame']
        #                         if frame_range[0] <= frame <= frame_range[1]:
        #                             # slice
        #                             slice = get_slice(label_gp, frame)
        #                             data = label_gp['data'][slice[0]:slice[1], :]
        #                             cols = label_gp['data'].attrs['columns'].split(';')
        #                             lookup = dict(zip(cols, range(len(cols))))
        #                             for i in range(data.shape[0]):
        #                                 label = Label()
        #                                 # class
        #                                 if 'labels_mapping' in label_gp:
        #                                     label.type = label_gp['labels_mapping'][data[i][lookup['Label']].astype(int)].decode('utf-8')
        #                                 else:
        #                                     label.type = label_gp['data_Label_mapping'][data[i][lookup['Label']].astype(int)].decode('utf-8')
        #                                 # position
        #                                 label.loc_3d[0] = data[i][lookup['XPos']]
        #                                 label.loc_3d[1] = data[i][lookup['YPos']]
        #                                 label.loc_3d[2] = data[i][lookup['ZPos']]
        #                                 # dimension (length, width, height)
        #                                 label.dim_3d[0] = data[i][lookup['XDim']]
        #                                 label.dim_3d[1] = data[i][lookup['YDim']]
        #                                 label.dim_3d[2] = data[i][lookup['ZDim']]
        #                                 # rotation as 3d matrix vectorized along columns
        #                                 # cx = math.cos(data['Roll'][i])
        #                                 # sx = math.sin(data['Roll'][i])
        #                                 # cy = math.cos(data['Pitch'][i])
        #                                 # sy = math.sin(data['Pitch'][i])
        #                                 yaw = data[i][lookup['Yaw']]
        #                                 cz = math.cos(yaw)
        #                                 sz = math.sin(yaw)
        #                                 # label.rot_3d[0, 0] = cy * cz
        #                                 # label.rot_3d[0, 1] = sx * sy * cz - cx * sz
        #                                 # label.rot_3d[0, 2] = cx * sy * cz + sx * sz
        #                                 # label.rot_3d[1, 0] = cy * sz
        #                                 # label.rot_3d[1, 1] = sx * sy * sz + cx * cz
        #                                 # label.rot_3d[1, 2] = cx * sy * sz - sx * cz
        #                                 # label.rot_3d[2, 0] = -sy
        #                                 # label.rot_3d[2, 1] = sx * cy
        #                                 # label.rot_3d[2, 2] = cx * cy
        #                                 label.rot_3d[0, 0] = cz
        #                                 label.rot_3d[0, 1] = -sz
        #                                 label.rot_3d[0, 2] = 0.0
        #                                 label.rot_3d[1, 0] = sz
        #                                 label.rot_3d[1, 1] = cz
        #                                 label.rot_3d[1, 2] = 0.0
        #                                 label.rot_3d[2, 0] = 0.0
        #                                 label.rot_3d[2, 1] = 0.0
        #                                 label.rot_3d[2, 2] = 1.0
        #                                 # velocity and acceleration
        #                                 label.vel_3d_filtered[0] = data[i][lookup['XVelFiltered']]
        #                                 label.vel_3d_filtered[1] = data[i][lookup['YVelFiltered']]
        #                                 label.vel_3d_filtered[2] = 0
        #                                 label.acc_3d_filtered[0] = data[i][lookup['XAccFiltered']]
        #                                 label.acc_3d_filtered[1] = data[i][lookup['YAccFiltered']]
        #                                 label.acc_3d_filtered[2] = 0
        #                                 if 'XVel' in lookup:
        #                                     label.vel_3d[0] = data[i][lookup['XVel']]
        #                                     label.vel_3d[1] = data[i][lookup['YVel']]
        #                                     label.vel_3d[2] = data[i][lookup['ZVel']]
        #                                 else:
        #                                     label.vel_3d = label.vel_3d_filtered.copy()
        #                                 if 'XAcc' in lookup:
        #                                     label.acc_3d[0] = data[i][lookup['XAcc']]
        #                                     label.acc_3d[1] = data[i][lookup['YAcc']]
        #                                     label.acc_3d[2] = data[i][lookup['ZAcc']]
        #                                 else:
        #                                     label.acc_3d = label.acc_3d_filtered.copy()
        #                                 # track id
        #                                 label.track_id = data_UUID_mapping[int(data[i][lookup['UUID']])] #int(data[i][lookup['UUID']])
        #                                 # no care flag
        #                                 if 'NoCare' in lookup:
        #                                     label.no_care = data[i][lookup['NoCare']]
        #                                 else:
        #                                     label.no_care = False
        #                                 if 'geom_NoCare' in lookup:
        #                                     label.geom_no_care = data[i][lookup['geom_NoCare']]
        #                                 else:
        #                                     label.geom_no_care = False
        #                                 # visibility
        #                                 vis_entries = [vv for vv in lookup if 'Visibility' in vv]
        #                                 if len(vis_entries) > 0:
        #                                     for ve in vis_entries:
        #                                         label.visibility = {int(ve.split('_')[1]): data[i][lookup[ve]]}
        #                                 # mark as annotation
        #                                 label.breed = 'annotation'
        #
        #                                 labels[lab_src].append(label)
        #                 # rename round truth source to 'GT'
        #                 if GT_src in labels:
        #                     label_GT_src = GT_src
        #                     labels['GT'] = labels.pop(GT_src)
        #                 # close file handle
        #                 label_file.close()

        # load cameras
        cameras = dict()
        camera_data = cbor_handler.get_data().get("camera", None)
        if "CAMERA" in data2load.keys() and camera_data:
            for cam_id, cam_name in dp_meta_data.aligned_sensor_names["CAMERA"].items():
                if dp_meta_data.aligned_fids["CAMERA"][cam_id] is not None:
                    if data2load["CAMERA"] == [] or cam_id in data2load["CAMERA"]:
                        if data_cache_relevant and data_cache[1].aligned_fids["CAMERA"][cam_id] == dp_meta_data.aligned_fids["CAMERA"][cam_id]:
                            cameras[cam_id] = data_cache[0].CAMERAS[cam_id]
                        else:
                            # get handle
                            cam_gp = camera_data[cam_name]
                            # get current frame
                            frame = dp_meta_data.aligned_fids["CAMERA"][cam_id]
                            # get image
                            # img = cam_gp['data'][frame]
                            img = cv2.imdecode(cam_gp["data"][frame], cv2.IMREAD_COLOR)
                            # if cam_gp['type'] == 'pandora-camera':
                            #     img = img[:, :, [2, 1, 0]]
                            # create camera structure
                            cameracalib = CameraCalib(
                                data={
                                    "img_width": cam_gp["calib"]["width"],
                                    "img_height": cam_gp["calib"]["height"],
                                    "T_V2C": cam_gp["calib"]["T_V2C"],
                                    "P": cam_gp["calib"]["P"],
                                }
                            )
                            # create camera object
                            cameras[cam_id] = Camera(
                                image=img,
                                camera_calib=cameracalib,
                                timestamp=dp_meta_data.aligned_timestamps["CAMERA"][cam_id],
                                frame_id=dp_meta_data.aligned_fids["CAMERA"][cam_id],
                                scan_id=dp_meta_data.aligned_sids["CAMERA"][cam_id],
                                sensor_name=dp_meta_data.aligned_sensor_names["CAMERA"][cam_id],
                            )

                            # load egomotion
                            ego_gr = cam_gp["ego_motion"]
                            cameras[cam_id].ego_transform = ego_gr["T0"][frame]
                            cameras[cam_id].ego_transform_to_previous = ego_gr["T"][frame]
                            cameras[cam_id].v_lon, cameras[cam_id].v_lat, cameras[cam_id].yawrate = ego_gr["ego_scalars"][frame]

        # create result structure
        ret = {
            "META": {
                "sensor_type": dp_meta_data.sensor_type,
                "sensor_id": dp_meta_data.sensor_id,
                "sensor_name": dp_meta_data.sensor_name,
                "sensor_frame_id": dp_meta_data.aligned_fids[dp_meta_data.sensor_type][dp_meta_data.sensor_id],
                "folder": dp_meta_data.folder,
                "sequence_name": dp_meta_data.sequence_name,
                "timestamp": dp_meta_data.aligned_timestamps[dp_meta_data.sensor_type][dp_meta_data.sensor_id],
                "GT_source": label_GT_src,
                "data_info": None,
                "vehicle_info": None,
            },
            "LIDARS": lidars,
            "RADARS": radars,
            "FUSEDRADARS": fusedradars,
            "CAMERAS": cameras,
            "LABEL": labels,
        }
        return ret

    def read_numpy_data_dict(self, data: dict):
        """
        directly read data from data dict

        Args:
            data: data dict to read from
        """
        # parse lidar
        lidar = dict()
        if "LIDAR" in data:
            lidar_pc = data["LIDAR"].astype(np.float32)
            lidar[0] = Lidar(lidar_pc, LidarCalib())

        # parse radar
        radar = dict()
        if "RADAR" in data:
            radar_pc = data["RADAR"].astype(np.float32)
            radar[0] = Radar(radar_pc, RadarCalib())

        # parse radar
        fusedradars = dict()
        if "FUSEDRADARS" in data:
            radar_pc = data["FUSEDRADARS"].astype(np.float32)
            fusedradars[0] = Radar(radar_pc, RadarCalib())

        # parse camera
        camera = dict()
        if "CAMERAS" in data:
            camera_img = data["CAMERAS"].astype(np.float32)
            camera[0] = Camera(camera_img, CameraCalib())

        # parse label
        label = dict()
        if "LABEL" in data:
            label = data["LABEL"]

        self.LIDARS = lidar
        self.RADARS = radar
        self.FUSEDRADARS = fusedradars
        self.CAMERAS = camera
        self.LABEL = label

    @staticmethod
    def quat2rot(q: np.array) -> np.array:
        """
        assistance for quaternions to rotation matrix

        Args:
            q: quaternion

        Returns:
            rotation matrix (4x4)
        """
        a, b, c, d = (q[0], q[1], q[2], q[3])
        rotation = np.array(
            [
                [1 - 2 * (c**2 + d**2), 2 * (b * c - a * d), 2 * (b * d + a * c)],
                [2 * (b * c + a * d), 1 - 2 * (b**2 + d**2), 2 * (c * d - a * b)],
                [2 * (b * d - a * c), 2 * (c * d + a * b), 1 - 2 * (b**2 + c**2)],
            ]
        )
        return rotation


def get_slice(dat: h5py.File, frm: int) -> list:
    """
    helper function to extract slice start and exclusive end

    Args:
        dat: h5py data handle
        frm: frame to get data slice for

    Returns:
        [slice start, slice end]
    """

    data_slices = dat["data_slices"]
    slice = [data_slices[frm, 0], 0]
    if frm < data_slices.shape[0] - 1:
        slice[1] = data_slices[frm + 1, 0]
    else:
        slice[1] = dat["data"].shape[0]
    return slice


class Data_package:
    """
    Assistance structure for ordering of unaligned data
    """

    # order of sensors
    sort_key_type_lookup = {"FUSEDRADAR": 0.01, "RADAR": 0.02, "CAMERA": 0.03, "LIDAR": 0.04}

    def __init__(self, typ: str, sensor_id: int, sensor_name: str, time: int, data_idx: int, scan_id: int, subsensors: Optional[dict] = None):
        """
        inits Data_package

        Args:
            typ:            sensor type
            sensor_id:      sensor id
            sensor_name:    sensor name
            time:           acquisition time
            data_idx:       data index
            scan_id:        scan id
            subsensors:     subsensor dictionary
        """
        self.type = typ
        self.sensor_id = sensor_id
        self.sensor_name = sensor_name
        self.time = time
        self.data_idx = data_idx
        self.scan_id = scan_id
        self.subsensors = subsensors

    def create_sort_key(self) -> float:
        """
        create sort key, lidar after camera after radar after unknown if same time is given
        Returns:
            sort key
        """
        return self.time + Data_package.sort_key_type_lookup.get(self.type, 0.0)


class DataProviderGenerator(object):
    """
    Generator module for data providers
    """

    def __enter__(self) -> "DataProviderGenerator":
        """
        implementation of __enter__ function

        Returns:
            self handle
        """
        return self

    def __exit__(self, exc_type: Optional[Type[BaseException]], exc_val: Optional[BaseException], exc_tb: Optional[TracebackType]):
        """
        implementation of __exit__ function

        Args:
            exc_type: indicates class of exception
            exc_val: indicates type of exception
            exc_tb: traceback is a report which has all of the information needed to solve the exception
        """
        # remove datasets from cbor handlers
        for dsets in self.datasets:
            CBOR_Handler.release_handler(dsets[0], dsets[1])

    def __init__(
        self,
        data_spec: List[DataProviderDataSpec],
        data2load: dict = {"LIDAR": [], "RADAR": [], "FUSEDRADAR": [], "CAMERA": [], "LABEL": []},
        type2generate: Type = DataProvider,
        cache_size: int = 0,
    ):
        """
        inits DataProviderGenerator

        Args:
            data_spec:      (List of) DataProviderDataSpec holding the data to load
            data2load:      dict of sensortypes holding lists of sensor ids for which data should be loaded, 'LABEL' with layers (e.g. 'GT') is also part
                            (empty list means all data is loaded, missing sensortype or None entry means no data is loaded)
            type2generate:  Data package class that shall be generated
            cache_size:     Storage in MB allowed for caching data
        """

        def init(data_spec: List[DataProviderDataSpec]):
            """
            nested init function

            Args:
                data_spec: (List of) DataProviderDataSpec holding the data to load
            """

            # store datasets used by this generator
            self.datasets = []

            # check type2generate
            if not issubclass(type2generate, DataProvider):
                raise ValueError("type2generate needs to be or derive from data_provider.DataProvider in DataProviderGenerator.__init__")
            self.type2generate = type2generate

            # store file packages selected for loading, spare sensor_type holding None, as data for this type shall not be loaded
            self.data2load = {sensor_type: id_list for sensor_type, id_list in data2load.items() if id_list is not None}

            # check input
            input_valid = False
            if type(data_spec) is DataProviderDataSpec:
                # either we have a DataProviderDataSpec (convert to one element list in this case)
                data_spec = [data_spec]
                input_valid = True
            elif type(data_spec) is list:
                # or a list of DataProviderDataSpec
                input_valid = True
                for dsp in data_spec:
                    if type(dsp) is not DataProviderDataSpec:
                        input_valid = False
                        break
            if not input_valid:
                raise ValueError(
                    "Input needs to be either data_provider.DataProviderDataSpec or list of  "
                    "data_provider.DataProviderDataSpec in DataProviderGenerator.__init__"
                )

            # store all source types in data_spec list
            self.source_types = []
            for dps in data_spec:
                if dps.type not in self.source_types:
                    self.source_types.append(dps.type)

            # check for forbidden 'trainingTool2' + FUSEDRADAR[0] configuration
            for st in self.source_types:
                if st in ["trainingTool2"]:
                    if "FUSEDRADAR" in self.data2load and 0 in self.data2load["FUSEDRADAR"]:
                        raise ValueError(
                            "FUSEDRADAR[0] is reserved for the artificial fused Radar sensor, thus its not allowed to load from"
                            " a sensor with id 0. Either load list of sensors you want to fuse (e.g. [1,3,5]),"
                            "or load all ([]), in DataProviderGenerator.__init__"
                        )

            # check data to load is int or in case of 'trainingTool2' it might be string
            for sensor_type in self.data2load.values():
                for sensor_id in sensor_type:
                    # in case only 'trainingTool2' is given, sensors might be given as strings or int
                    if self.source_types == ["trainingTool2"]:
                        if type(sensor_id) not in [int, str]:
                            raise ValueError(
                                'data2load sensor entries must be empty lists or lists of type int (sensor_id), (or in case of "trainingTool2" optionally of type str (sensor_name)), in DataProviderGenerator.__init__'
                            )
                    else:
                        # only int is allowed otherwise
                        if type(sensor_id) is not int:
                            raise ValueError(
                                'data2load sensor entries must be empty lists or lists of type int (sensor_id), (or in case of "trainingTool2" of type str (sensor_name), optionally), in DataProviderGenerator.__init__'
                            )

            # check for data and create lookups and meta structures
            self.data_spec = data_spec
            self.file_meta_data_list = None
            self.sequence_lookup = None
            self.triggerable = dict()
            self.radar_versions = ["v0"]
            self.current_radar_version_index = 0
            self.create_file_meta_data()

            # init skip list, this list is either None, or a boolean list of size of self.trigger2total, which tells generator to skip data_packages at
            # at indices holding False, calling self.set_data2trigger will rest it to None to keep sizes consistant
            self.skip_list = None

            # initially trigger on everything
            self.trigger2total = []
            self.total2trigger = []
            self.trigger_samples_per_sequence = dict()
            self.first_trigger_sample_per_sequence = dict()
            self.set_data2trigger(self.triggerable)

            # init label converter empty or by files if specified in dataspecs
            self.label_converter = ClassMapper()
            for dps in data_spec:
                if dps.mapping_file not in [None, ""]:
                    self.label_converter.update(dps.mapping_file)

            # init data cache
            self.data_cache_size_cur = 0
            self.data_cache = dict()
            self.data_cache_queue = collections.deque()
            if cache_size > 0:
                self.enable_data_cache(cache_size)
            else:
                self.disable_data_cache()

            # init data manipulator list
            self.data_manipulators = []

        init(data_spec)

    def clear_data_cache(self):
        """
        clear data cache
        """
        self.data_cache_size_cur = 0
        self.data_cache.clear()
        self.data_cache_queue.clear()

    def enable_data_cache(self, cache_size: int):
        """
        enable data cache

        Args:
            cache_size: cache size in megabytes
        """
        self.data_cache_size_max = cache_size * 1000000
        self.clear_data_cache()

    def disable_data_cache(self):
        """
        disable data cache
        """
        self.data_cache_size_max = 0
        self.clear_data_cache()

    def generate(self, sample_number: int) -> Optional[Type]:
        """
        generate data_provider package for sample number

        Returns:
            data sample
        """
        if sample_number >= 0 and sample_number < self.size_trigger():

            # check if frame shall be skipped
            if self.skip_list is not None and self.skip_list[sample_number] is False:
                print("Frame %d is skipped due to skip_list" % sample_number)
                return "SKIPPED"

            # get sample number it total list
            total_sample_number = self.trigger2total[sample_number]

            # caching
            if self.data_cache_size_max > 0:
                # with caching, query cache for sample
                dp_cache_obj = self.data_cache.get(total_sample_number, None)
                if dp_cache_obj is None:
                    # add last sample to allow to copy reused data and spare disk loading
                    if len(self.data_cache_queue) > 0:
                        data_cache = self.data_cache[self.data_cache_queue[-1]]
                    else:
                        data_cache = None
                    # create sample
                    dp_cache = self.type2generate(
                        dp_meta_data=self.file_meta_data_list[total_sample_number],
                        tag=self.get_tag(sample_number),
                        data2load=self.data2load,
                        label_converter=self.label_converter,
                        data_cache=data_cache,
                        version2load={"RADAR": self.get_current_radar_version(), "FUSEDRADAR": self.get_current_radar_version()},
                        data_manipulators=self.data_manipulators,
                    )
                    # insert into cache
                    dp_size = dp_cache.get_byte_size()
                    self.data_cache_size_cur += dp_size
                    self.data_cache[total_sample_number] = (dp_cache, self.file_meta_data_list[total_sample_number], dp_size)
                    self.data_cache_queue.append(total_sample_number)
                    # do not exceed cache size
                    while self.data_cache_size_cur > self.data_cache_size_max:
                        self.data_cache_size_cur -= self.data_cache[self.data_cache_queue[0]][2]
                        del self.data_cache[self.data_cache_queue[0]]
                        self.data_cache_queue.popleft()
                else:
                    # get from cache
                    dp_cache = dp_cache_obj[0]
                    # call cache independent data manipulations
                    dp_cache.update_data_manipulators_cache_independent(
                        data_cache=dp_cache_obj,
                        registered_data_manipulators=self.data_manipulators,
                        data2load=self.data2load,
                        version2load={"RADAR": self.get_current_radar_version(), "FUSEDRADAR": self.get_current_radar_version()},
                        label_converter=self.label_converter,
                        data_handlers=self.file_meta_data_list[total_sample_number].data_handlers,
                    )

                # copy data from cache to prevent user changes cache copy down the pipeline
                dp = copy.deepcopy(dp_cache)
            else:
                # no caching, generate new sample
                dp = self.type2generate(
                    dp_meta_data=self.file_meta_data_list[total_sample_number],
                    tag=self.get_tag(sample_number),
                    data2load=self.data2load,
                    label_converter=self.label_converter,
                    version2load={"RADAR": self.get_current_radar_version(), "FUSEDRADAR": self.get_current_radar_version()},
                    data_manipulators=self.data_manipulators,
                )

            # # print sample info
            # ts = None
            # if self.file_meta_data_list[total_sample_number].aligned_timestamps is not None:
            #     ts = self.file_meta_data_list[total_sample_number].aligned_timestamps[self.file_meta_data_list[total_sample_number].sensor_type][self.file_meta_data_list[total_sample_number].sensor_id]
            # print('GEN', total_sample_number, self.get_tag(sample_number), ts, '   CACHE_SIZE:', len(self.data_cache), self.data_cache_size_cur)

            # return data provider
            return dp
        else:
            return None

    def set_skiplist(self, skip_list: Optional[list]):
        """
        set skiplist to tell generator to ignore certain frames

        Args:
            skip_list: boolean list of size trigger2total holding False for frame ids to skip, None to reset
        """
        # check input
        if skip_list is not None:
            for val in skip_list:
                if type(val) is not bool:
                    raise ValueError("Input <skip_list> must be either None or list of bool in DataProviderGenerator.set_skiplist")

            # check length
            if len(skip_list) != len(self.trigger2total):
                raise ValueError("Length of list <skip_list> must match number of tigger frames in DataProviderGenerator.set_skiplist")

        # set skip_list
        self.skip_list = skip_list

    def set_data2trigger(self, data2trigger: Optional[Dict[str, List]] = None):
        """
        set trigger list

        Args:
            data2trigger: if not None: dict of sensortypes holding lists of sensor ids for which data should be triggered
                          (empty list means all sensor ids trigger)
                          'trigger' means a data package is generated, 'loaded' means data is added to a package of the next triggered sensor package
                          if None: set to all triggerable data
        """
        # reset if input is None
        if data2trigger is None:
            self.set_data2trigger(self.triggerable)

        # reset skiplist in case it was set, to prevent inconsistancy in frame counting
        if self.skip_list is not None:
            self.skip_list = None
            print("WARNING: Skiplist reseted due to call of set_data2trigger!")

        # check sanity of data2load and data2trigger (no trigger which is not loaded)
        for sensor_type, sensor_ids in data2trigger.items():
            if sensor_type not in self.data2load:
                raise ValueError("Each sensor type in data2trigger must be present in data2load DataProviderGenerator.set_data2trigger")
            if sensor_ids == [] and self.data2load[sensor_type] != []:
                if sensor_type == "FUSEDRADAR" and "trainingTool2" in self.source_types and "FUSEDRADAR" in self.data2load:
                    # in case radar is fused to a single sensor with id 0, we can trigger on fusedradar if its available in data2load independent on which senseors it was fused from
                    pass
                else:
                    raise ValueError(
                        "%s triggers on all sensor ids (data2trigger[%s]=[]), so all data must be loaded as well (data2load[%s]=[]) in data2load DataProviderGenerator.set_data2trigger"
                        % (sensor_type, sensor_type, sensor_type)
                    )
            for id in sensor_ids:
                if self.data2load[sensor_type] != [] and id not in self.data2load[sensor_type]:
                    if (
                        sensor_type == "FUSEDRADAR"
                        and (sensor_ids == [] or sensor_ids == [0])
                        and "trainingTool2" in self.source_types
                        and "FUSEDRADAR" in self.data2load
                    ):
                        # in case radar is fused to a single sensor with id 0, we can trigger on fusedradar if its available in data2load independent on which senseors it was fused from
                        pass
                    else:
                        raise ValueError(
                            "%s triggers on sensor id %s, so id must be loaded as well (in data2load[%s]) in DataProviderGenerator.set_data2trigger"
                            % (sensor_type, id, sensor_type)
                        )

        # check against triggerable
        self.data2trigger = dict()
        for key, dat in data2trigger.items():
            # check sensor
            if key not in self.triggerable:
                print("WARNING: Sensor %s is not triggerable in this dataset" % key)
            else:
                # add sensor
                if key not in self.data2trigger:
                    self.data2trigger[key] = []
                # add all sensor ids
                if dat == []:
                    self.data2trigger[key] = self.triggerable[key]
                else:
                    # add single sensor ids
                    for id in dat:
                        if id not in self.triggerable[key]:
                            print("WARNING: Sensor %s id %d is not triggerable in this dataset" % (key, id))
                        else:
                            self.data2trigger[key].append(id)

        # change data lookup
        self.trigger2total = []
        self.total2trigger = [None] * self.size_total()
        self.trigger_samples_per_sequence = dict()
        self.first_trigger_sample_per_sequence = dict()
        trigger_cnt = 0
        for i in range(len(self.file_meta_data_list)):
            if (
                self.file_meta_data_list[i].sensor_type in self.data2trigger
                and self.file_meta_data_list[i].sensor_id in self.data2trigger[self.file_meta_data_list[i].sensor_type]
            ):
                # fill lookup lists
                self.trigger2total.append(i)
                self.total2trigger[i] = trigger_cnt
                # increase counters
                if self.file_meta_data_list[i].sequence_name not in self.trigger_samples_per_sequence:
                    self.trigger_samples_per_sequence[self.file_meta_data_list[i].sequence_name] = 0
                    self.first_trigger_sample_per_sequence[self.file_meta_data_list[i].sequence_name] = trigger_cnt
                self.trigger_samples_per_sequence[self.file_meta_data_list[i].sequence_name] += 1
                trigger_cnt += 1

    def get_trigger_sample_number(self, sequence: Optional[str]) -> int:
        """
        get number of trigger samples for given sequence (get total number if None is entered)

        Args:
            sequence: sequence to check, if None: add up samples of all sequences

        Returns:
            number of trigger samples
        """
        if sequence is None:
            cnt = 0
            for tsps in self.trigger_samples_per_sequence.value():
                cnt += tsps
            return cnt
        else:
            return self.trigger_samples_per_sequence.get(sequence, 0)

    def get_first_trigger_sample_id(self, sequence: str) -> Optional[int]:
        """
        get first trigger sample id of a sequence

        Args:
            sequence: sequence name

        Returns:
            first trigger sample if exists, None otherwise
        """
        return self.first_trigger_sample_per_sequence.get(sequence, None)

    def get_id_total_from_trigger(self, idx: int) -> Optional[int]:
        """
        get total package list id from current trigger list id, return None when out of bounds

        Args:
            idx: trigger id

        Returns:
            aligned total package list id if exists, None otherwise
        """
        if idx < 0 or idx >= self.size_trigger():
            return None
        else:
            return self.trigger2total[idx]

    def get_id_trigger_from_total_next(self, idx: int) -> Optional[int]:
        """
        get next valid trigger package list id from current total list id, return None when out of bounds

        Args:
            idx: total list id

        Returns:
            next valid trigger id if exists, None otherwise
        """
        if idx < 0 or idx >= self.size_total():
            return None
        while self.total2trigger[idx] is None:
            idx += 1
            if idx < 0 or idx >= self.size_total():
                return None
        return self.total2trigger[idx]

    def get_id_trigger_from_total_previous(self, idx: int) -> Optional[int]:
        """
        get previous valid trigger package list id from current total list id, return None when out of bounds

        Args:
            idx: total list id

        Returns:
            previous valid trigger id if exists, None otherwise
        """
        if idx < 0 or idx >= self.size_total():
            return None
        while self.total2trigger[idx] is None:
            idx -= 1
            if idx < 0 or idx >= self.size_total():
                return None
        return self.total2trigger[idx]

    def get_first_trigger_id_of_next_sequence(self, query_id: int) -> Optional[int]:
        """
        get first trigger id of the next sequence in loaded data list

        Args:
            query_id: query id

        Returns:
            first trigger id of the next sequence if exists, None otherwise
        """
        # check input validity
        if query_id < 0 or query_id >= self.size_trigger() - 1:
            return None
        # search for id of sample from other sequence
        cur_seq = self.file_meta_data_list[self.trigger2total[query_id]].sequence_number
        for id in range(query_id + 1, self.size_trigger()):
            total_id = self.trigger2total[id]
            if self.file_meta_data_list[total_id].sequence_number != cur_seq:
                return id
        # did not find further sequences
        return None

    def get_first_trigger_id_of_previous_sequence(self, query_id: int) -> Optional[int]:
        """
        get first trigger id of the previous sequence in loaded data list

        Args:
            query_id: query id

        Returns:
            irst trigger id of the previous sequence if exists, None otherwise
        """
        # check input validity
        if query_id < 1 or query_id >= self.size_trigger():
            return None
        # search for id of sample from other sequence
        cur_seq = self.file_meta_data_list[self.trigger2total[query_id]].sequence_number
        prev_seq = None
        for id in range(query_id, -1, -1):
            total_id = self.trigger2total[id]
            if self.file_meta_data_list[total_id].sequence_number != cur_seq:
                prev_seq = self.file_meta_data_list[total_id].sequence_number
                break
        # search for first trigger id with found sequence
        if prev_seq is not None:
            for id in self.sequence_lookup[prev_seq]:
                if self.total2trigger[id] is not None:
                    return self.total2trigger[id]
        # did not find further sequences
        return None

    def get_triggerable(self) -> Dict[str, List]:
        """
        get dict of triggerable sensors

        Returns:
            dict of triggerable sensors {sensor_type: list of sensor_ids}
        """
        return copy.deepcopy(self.triggerable)

    def get_current_triggers(self) -> Dict[str, List]:
        """
        get current trigger sensors

        Returns:
            current trigger sensors {sensor_type: list of sensor_ids}
        """
        return copy.deepcopy(self.data2trigger)

    def add_static_class_mapping(self, src: str, dst: str, no_care: bool = False):
        """
        add a static class mapping rule

        Args:
            src:        source class
            dst:        target class
            no_care:    no_care flag for target class
        """
        self.label_converter.add_static_mapping(src=src, dst=dst, no_care=no_care)

    def add_dynamic_class_mapping(
        self, src: str, speed_threshold, dst_moving: str, dst_stationary, no_care_moving: bool = False, no_care_stationary: bool = False
    ):
        """
        add a dynamic class mapping rule

        Args:
            src:                source class
            speed_threshold:    threshold in m/s for absolute speed to decide between moving or stationary class mapping
            dst_moving:         target class for moving
            dst_stationary:     target class for stationary
            no_care_moving:     no_care flag for moving target class
            no_care_stationary: no_care flag for stationary target class
        """
        self.label_converter.add_dynamic_mapping(
            src=src,
            speed_threshold=speed_threshold,
            dst_moving=dst_moving,
            dst_stationary=dst_stationary,
            no_care_moving=no_care_moving,
            no_care_stationary=no_care_stationary,
        )

    def set_label_converter(self, lab_conv: Dict[str, str]):
        """
        clear and set a label converter (add dict to static label mapping)

        Args:
            lab_conv: dict holding the mappings to add
        """
        self.label_converter.reset()
        for src, dst in lab_conv.items():
            self.label_converter.add_static_mapping(src, dst)

    def get_label_converter(self) -> ClassMapper:
        """
        get a copy of the label converter

        Returns:
            class mapper handle
        """
        return copy.deepcopy(self.label_converter)

    def get_full_sample_tag_data(self, sample_number: int) -> Optional[DataProviderMetaData]:
        """
        get meta data list entry

        Args:
            sample_number: query sample number

        Returns:
            file meta data if exists, None otherwise
        """
        if sample_number >= 0 and sample_number < self.size_trigger():
            total_sample_number = self.trigger2total[sample_number]
            return self.file_meta_data_list[total_sample_number]
        else:
            return None

    def get_tag(self, sample_number: int) -> Optional[str]:
        """
        get sample tag (identifier created from sequence name and file id)

        Args:
            sample_number: query sample number

        Returns:
            sample tag if exists, None otherwise
        """
        if sample_number >= 0 and sample_number < self.size_trigger():
            total_sample_number = self.trigger2total[sample_number]
            fid = (
                self.file_meta_data_list[total_sample_number].sequence_name
                + "_"
                + self.file_meta_data_list[total_sample_number].sensor_type
                + "_"
                + str(self.file_meta_data_list[total_sample_number].sensor_id)
                + "_"
            )
            if type(self.file_meta_data_list[total_sample_number].aligned_fids) is int:
                fid += str(self.file_meta_data_list[total_sample_number].aligned_fids)
            else:
                fid += str(
                    self.file_meta_data_list[total_sample_number].aligned_fids[self.file_meta_data_list[total_sample_number].sensor_type][
                        self.file_meta_data_list[total_sample_number].sensor_id
                    ]
                )
            return fid
        else:
            return None

    def size_total(self) -> int:
        """
        get total number of data packages

        Returns:
            total number of sample packages
        """
        return len(self.file_meta_data_list)

    def size_trigger(self) -> int:
        """
        get current number of triggerable datapackages

        Returns:
            current number of triggerable datapackages
        """
        return len(self.trigger2total)

    def get_first_trigger_number_after_time(self, log_name: str, timestamp: float) -> Optional[int]:
        """
        get trigger number next after timestamp for sequence

        Args:
            log_name:   sequence name
            timestamp:  query timestamp

        Returns:
            first trigger sample after requested timestamp if exists, None otherwise
        """
        # get first trigger data package equal or above querytimestamp
        for i in range(len(self.trigger2total)):
            meta_data = self.file_meta_data_list[self.trigger2total[i]]
            if meta_data.sequence_name == log_name:
                for trigger_sensor, trigger_ids in self.data2trigger.items():
                    sensor_times = meta_data.aligned_timestamps[trigger_sensor]
                    for sensor_id, time in sensor_times.items():
                        if trigger_ids == [] or sensor_id in trigger_ids:
                            if timestamp <= time:
                                return i
        # if none was found return None
        return None

    def create_file_meta_data(self):
        """
        get list of file meta data located in specified database
        """
        # cycle over list of inputs
        self.file_meta_data_list = []
        sequence_count = 0
        for log_spec in [ls for ds in self.data_spec for ls in ds.get_log_specs()]:
            if log_spec["log_type"] in ["trainingTool2"]:
                # deal with hdf5 data
                sequence_count = self.create_meta_data_from_hdf5(
                    log_spec["log_type"], log_spec["log_folder"], log_spec["log_name"], log_spec["label_source"], sequence_count
                )
            elif log_spec["log_type"] in ["cbor"]:
                # deal with cbor data
                sequence_count = self.create_meta_data_from_cbor(
                    log_spec["log_type"], log_spec["log_folder"], log_spec["log_name"], log_spec["label_source"], sequence_count
                )
            else:
                raise ValueError("Unknown source type %s in DataProviderGenerator.create_file_meta_data" % log_spec["log_type"])
        # add None to radar versions
        self.radar_versions.append(None)

        # check if something was found and raise error
        if len(self.file_meta_data_list) == 0:
            self.file_meta_data_list = None
        if self.file_meta_data_list is None:
            raise ValueError("No valid files found for specified source type and folder")

        # create subfolder lookup, fill framenumbers and identify triggerable sensors
        self.sequence_lookup = dict()
        last_seq = -1
        for i in range(len(self.file_meta_data_list)):
            meta = self.file_meta_data_list[i]
            # if sequence changes, add new entry
            if meta.sequence_number != last_seq:
                last_seq = meta.sequence_number
                self.sequence_lookup[meta.sequence_number] = [i]
                sequence_counter = 0
            else:
                self.sequence_lookup[meta.sequence_number].append(i)

            # replace file counter in meta data list
            self.file_meta_data_list[i] = self.file_meta_data_list[i]._replace(framenumber=sequence_counter)
            sequence_counter += 1

            # check triggerable and add if not yet done
            if self.file_meta_data_list[i].sensor_type not in self.triggerable:
                self.triggerable[self.file_meta_data_list[i].sensor_type] = []
            if self.file_meta_data_list[i].sensor_id not in self.triggerable[self.file_meta_data_list[i].sensor_type]:
                self.triggerable[self.file_meta_data_list[i].sensor_type].append(self.file_meta_data_list[i].sensor_id)

    @staticmethod
    def natural_sort(l: List[str]) -> List[str]:
        """
        natural sort

        Args:
            l: list to sort

        Returns:
            sorted list
        """
        convert = lambda text: int(text) if text.isdigit() else text.lower()
        alphanum_key = lambda key: [convert(c) for c in re.split("([0-9]+)", key)]
        return sorted(l, key=alphanum_key)

    """
    create meta data for hdf5 trainingtool2 data
    """

    def create_meta_data_from_hdf5(self, styp: str, sloc: str, snam: str, lsrc: str, sequence_count: int) -> int:
        """
        create meta data for hdf5 trainingtool2 data

        Args:
            styp:           sequence data type (e.g.: 'trainingTool2')
            sloc:           data folder
            snam:           sequence name
            lsrc:           GT label source
            sequence_count: current number of sequences

        Returns:
            new number of sequences
        """
        # load sensor mappings
        lidar_file_name = sloc + "/Lidar/" + snam + ".h5"
        meta_file_name = sloc + "/Meta/" + snam + ".h5"
        radar_detections_file_name = sloc + "/Radar_Detections/" + snam + ".h5"
        camera_file_name = sloc + "/Camera/" + snam + ".h5"
        # cam_time_files = [sloc + '/Camera%dFrameTime__v0/' % c + sn + '/frame_time.csv' for c in range(5)]

        # create data package list
        data_packages = []
        # use offset corrections to put lidar timestamps to front pass to minimize difference between lidar and radar (and align camera)
        offset_correction_lidar = 0  # 50
        # parse cameras
        if "CAMERA" in self.data2load:
            if os.path.exists(camera_file_name):
                camera_file = h5py.File(camera_file_name, "r")
                # resolve sensor name entries in data2load
                self.resolve_str_in_data2load(camera_file["camera_idx_to_name"], "CAMERA")
                # get data
                for cam_id, cam_name in enumerate(camera_file["camera_idx_to_name"]):
                    if self.data2load["CAMERA"] == [] or cam_id in self.data2load["CAMERA"]:
                        cam_gp = camera_file["sensors"][cam_name]
                        time = cam_gp["timestamps"][:, 0] * 1000
                        cam_name = cam_name.decode("UTF-8")
                        data_packages += [
                            Data_package(typ="CAMERA", sensor_id=cam_id, sensor_name=cam_name, time=time[i], data_idx=i, scan_id=None)
                            for i in range(time.shape[0])
                        ]
                # close file handles
                camera_file.close()
            else:
                print("\033[93mWARNING: No camera file found for log %s!\033[0m" % snam)

        # parse lidar
        if "LIDAR" in self.data2load:
            lidar_file = h5py.File(lidar_file_name, "r")
            # resolve sensor name entries in data2load
            self.resolve_str_in_data2load(lidar_file["lidar_idx_to_name"], "LIDAR")
            # get data
            for lid_id, lid_name in enumerate(lidar_file["lidar_idx_to_name"]):
                if self.data2load["LIDAR"] == [] or lid_id in self.data2load["LIDAR"]:
                    lid_gp = lidar_file["sensors"][lid_name]
                    lid_name = lid_name.decode("UTF-8")
                    time = lid_gp["timestamps"][:, 0] * 1000 + offset_correction_lidar
                    data_packages += [
                        Data_package(typ="LIDAR", sensor_id=lid_id, sensor_name=lid_name, time=time[i], data_idx=i, scan_id=None) for i in range(time.shape[0])
                    ]
            # close file handles
            lidar_file.close()

        # parse fused radar
        if "FUSEDRADAR" in self.data2load:
            meta_file = h5py.File(meta_file_name, "r")
            if os.path.exists(radar_detections_file_name):
                radar_file = h5py.File(radar_detections_file_name, "r")
                batch2radar_map = radar_file["batch_to_radar_frame_map"]
            else:
                radar_file = None
                batch2radar_map = None
            # resolve sensor name entries in data2load
            self.resolve_str_in_data2load(meta_file["radar_idx_to_name"], "FUSEDRADAR")
            # load meta data
            time = meta_file["timestamps"][:, 0] * 1000
            scan_id = meta_file["scan_id"][:, 0]
            # get data
            for i in range(meta_file["timestamps"].shape[0]):
                # add subsensor information
                sub_sensor_packages = dict()
                for sd, rad_name in enumerate(meta_file["radar_idx_to_name"]):
                    if self.data2load["FUSEDRADAR"] == [] or sd in self.data2load["FUSEDRADAR"]:
                        subsensor_scan_id = meta_file["scan_id"][i, sd]
                        if subsensor_scan_id >= 0:
                            radar_frm = batch2radar_map[i, sd] if radar_file else -1
                            sub_sensor_packages[sd] = {
                                "sensor_name": rad_name.decode("UTF-8"),
                                "timestamp": meta_file["radar_timestamps"][i, sd] * 1000,
                                "frame_id": radar_frm,
                                "scan_id": subsensor_scan_id,
                            }
                        else:
                            sub_sensor_packages[sd] = None

                # add data package
                data_packages.append(
                    Data_package(
                        typ="FUSEDRADAR", sensor_id=0, sensor_name="Fused", time=time[i], data_idx=i, scan_id=scan_id[i], subsensors=sub_sensor_packages
                    )
                )
            # close file handles
            if radar_file:
                radar_file.close()
            meta_file.close()
        # parse radar
        if "RADAR" in self.data2load:
            radar_file = h5py.File(radar_detections_file_name, "r")
            # resolve sensor name entries in data2load
            self.resolve_str_in_data2load(radar_file["radar_idx_to_name"], "RADAR")
            # get data
            for rad_id, rad_name in enumerate(radar_file["radar_idx_to_name"]):
                if self.data2load["RADAR"] == [] or rad_id in self.data2load["RADAR"]:
                    rad_gp = radar_file["sensors"][rad_name]
                    name = rad_name.decode("UTF-8")
                    time = rad_gp["timestamps"][:, 0] * 1000
                    scan_id = rad_gp["scan_id"][:, 0]
                    for i in range(rad_gp["timestamps"].shape[0]):
                        data_packages.append(Data_package(typ="RADAR", sensor_id=rad_id, sensor_name=name, time=time[i], data_idx=i, scan_id=scan_id[i]))
            # close file handles
            radar_file.close()

        # get sensor map
        sensor_map = {"LABEL": None}
        timestamp_map = {}
        subsensor_map = {}
        scanid_map = {}
        sensor_name_map = {}
        for dp in data_packages:
            if dp.type not in sensor_map:
                sensor_map[dp.type] = {}
                timestamp_map[dp.type] = {}
                subsensor_map[dp.type] = {}
                scanid_map[dp.type] = {}
                sensor_name_map[dp.type] = {}
            if dp.sensor_id not in sensor_map[dp.type]:
                sensor_map[dp.type][dp.sensor_id] = None
                timestamp_map[dp.type][dp.sensor_id] = None
                subsensor_map[dp.type][dp.sensor_id] = None
                scanid_map[dp.type][dp.sensor_id] = None
                sensor_name_map[dp.type][dp.sensor_id] = None

        # sort by timestamps
        data_packages.sort(key=lambda x: x.create_sort_key())

        # feed data packages to meta data list and fill aligned data
        framenumber = 0
        last_label_location = None
        for dp in data_packages:
            # update maps
            timestamp_map[dp.type][dp.sensor_id] = dp.time
            sensor_map[dp.type][dp.sensor_id] = dp.data_idx
            subsensor_map[dp.type][dp.sensor_id] = dp.subsensors
            scanid_map[dp.type][dp.sensor_id] = dp.scan_id
            sensor_name_map[dp.type][dp.sensor_id] = dp.sensor_name
            # create respective labelfile name
            if "LABEL" in self.data2load:
                if dp.type == "FUSEDRADAR":
                    sensor_map["LABEL"] = {"file": sloc + "/Labels/" + snam + ".h5", "frame": dp.data_idx, "source": lsrc, "sensor": dp.sensor_name}
                    last_label_location = sensor_map["LABEL"]
                elif dp.type == "LIDAR":
                    # sensor_map['LABEL'] = last_label_location
                    sensor_map["LABEL"] = {"file": sloc + "/Labels_Sensorwise/" + snam + ".h5", "frame": dp.data_idx, "source": lsrc, "sensor": dp.sensor_name}
                    last_label_location = sensor_map["LABEL"]
                elif dp.type == "CAMERA":
                    sensor_map["LABEL"] = last_label_location
                elif dp.type == "RADAR":
                    sensor_map["LABEL"] = {"file": sloc + "/Labels_Sensorwise/" + snam + ".h5", "frame": dp.data_idx, "source": lsrc, "sensor": dp.sensor_name}
                    last_label_location = sensor_map["LABEL"]

            # create meta data entries, field framenumber will be replaced afterwards
            self.file_meta_data_list.append(
                DataProviderMetaData(
                    sensor_type=dp.type,
                    sensor_id=dp.sensor_id,
                    sensor_name=dp.sensor_name,
                    source_type=styp,
                    folder=sloc,
                    sequence_name=snam,
                    sequence_number=sequence_count,
                    aligned_sensor_names=copy.deepcopy(sensor_name_map),
                    aligned_fids=copy.deepcopy(sensor_map),
                    aligned_timestamps=copy.deepcopy(timestamp_map),
                    aligned_sids=copy.deepcopy(scanid_map),
                    aligned_subsensors=copy.deepcopy(subsensor_map),
                    framenumber=framenumber,
                    data_handlers=None,
                    flags=dict(),
                )
            )
            # increase frame counter
            framenumber += 1

        # increment sequence counter
        sequence_count += 1

        # report current sequence counter
        return sequence_count

    def create_meta_data_from_cbor(self, styp: str, sloc: str, snam: str, lsrc: str, sequence_count: int) -> int:
        """
        create meta data for cbor data

        Args:
            styp:           sequence data type (e.g.: 'cbor')
            sloc:           data folder
            snam:           sequence name
            lsrc:           GT label source
            sequence_count: current number of sequences

        Returns:
            new number of sequences
        """
        # remember handlers to release them when Generator is closed
        self.datasets.append((sloc, snam))
        # parse cbor file
        cbor_handler = CBOR_Handler.get_handler(sloc, snam)

        # create data package list
        data_packages = []
        # use offset corrections to put lidar timestamps to front pass to minimize difference between lidar and radar (and align camera)
        offset_correction_lidar = 0  # 50
        # parse cameras
        if "CAMERA" in self.data2load:
            camera_data = cbor_handler.get_data().get("camera", None)
            if camera_data:
                # resolve sensor name entries in data2load
                camera_idx_to_name = [None] * len(camera_data)
                for cd in camera_data.values():
                    camera_idx_to_name[cd["internal-identifier"]] = cd["name"]
                self.resolve_str_in_data2load(camera_idx_to_name, "CAMERA")
                # get data
                for cam_id, cam_name in enumerate(camera_idx_to_name):
                    if self.data2load["CAMERA"] == [] or cam_id in self.data2load["CAMERA"]:
                        cam_gp = camera_data[cam_name]
                        time = np.array(cam_gp["timestamps"]) * 1000
                        data_packages += [
                            Data_package(typ="CAMERA", sensor_id=cam_id, sensor_name=cam_name, time=time[i], data_idx=i, scan_id=None)
                            for i in range(time.shape[0])
                        ]
            else:
                print("\033[93mWARNING: No camera file found for log %s!\033[0m" % snam)

        # parse lidar
        if "LIDAR" in self.data2load:
            lidar_data = cbor_handler.get_data().get("lidar", None)
            if lidar_data:
                # resolve sensor name entries in data2load
                lidar_idx_to_name = sorted(lidar_data.keys())
                self.resolve_str_in_data2load(lidar_idx_to_name, "LIDAR")
                # get data
                for lid_id, lid_name in enumerate(lidar_idx_to_name):
                    if self.data2load["LIDAR"] == [] or lid_id in self.data2load["LIDAR"]:
                        lid_gp = lidar_data[lid_name]
                        time = np.array(lid_gp["timestamps"]) * 1000 + offset_correction_lidar
                        data_packages += [
                            Data_package(typ="LIDAR", sensor_id=lid_id, sensor_name=lid_name, time=time[i], data_idx=i, scan_id=None)
                            for i in range(time.shape[0])
                        ]

        # parse fused radar
        if "FUSEDRADAR" in self.data2load:
            network_data = cbor_handler.get_data().get("network_input", None)
            radar_data = cbor_handler.get_data().get("radar", None)
            if network_data and radar_data and "network" in network_data:
                # get handles
                metadata = network_data["network"]["metadata"]
                # resolve sensor name entries in data2load
                radar_idx_to_name = [None] * len(radar_data)
                for rd in radar_data.values():
                    radar_idx_to_name[rd["internal-identifier"]] = rd["name"]
                self.resolve_str_in_data2load(radar_idx_to_name, "FUSEDRADAR")
                batch2radar_map = -np.ones([len(metadata), len(radar_idx_to_name)], dtype=int)
                max_scan_id = 2**16 - 1
                scan_ids = np.array([md["scan_idx"] for md in metadata])
                assert np.max(scan_ids) <= max_scan_id, "Scan IDs are expected to be 16bit only"
                scan_ids_maxed = np.maximum.accumulate(scan_ids, axis=0)
                scan_ids_extended = scan_ids.copy()
                scan_ids_extended[np.logical_and(scan_ids_extended < scan_ids_maxed, scan_ids_extended != -1)] += max_scan_id + 1
                fr_timestamps = network_data["network"]["timestamps"]
                for rad_id, rad_name in enumerate(radar_idx_to_name):
                    # get left neighbors in time for each "radar batch" timestamp
                    rad_scan_ids = np.array(radar_data[rad_name]["scan_id"])
                    assert np.max(rad_scan_ids) <= max_scan_id, "Scan IDs are expected to be 16bit only"
                    valid = scan_ids[:, rad_id] != -1
                    rad_scan_ids_maxed = np.maximum.accumulate(rad_scan_ids)
                    rad_scan_ids_expanded = rad_scan_ids.copy()
                    rad_scan_ids_expanded[np.logical_and(rad_scan_ids < rad_scan_ids_maxed, rad_scan_ids != -1)] += max_scan_id + 1
                    batch2radar_map[valid, rad_id] = np.searchsorted(rad_scan_ids_expanded, scan_ids_extended[valid, rad_id])

                # load batch to radar frame mapping
                time = np.array(fr_timestamps) * 1000
                scan_id = scan_ids[:, 0]
                # get data
                for i in range(len(fr_timestamps)):
                    # add subsensor information
                    sub_sensor_packages = dict()
                    for sd, rad_name in enumerate(radar_idx_to_name):
                        if self.data2load["FUSEDRADAR"] == [] or sd in self.data2load["FUSEDRADAR"]:
                            if batch2radar_map[i, sd] >= 0:
                                radar_frm = batch2radar_map[i, sd]
                                sub_sensor_packages[sd] = {
                                    "sensor_name": rad_name,
                                    "timestamp": radar_data[rad_name]["timestamps"][radar_frm] * 1000,
                                    "frame_id": radar_frm,
                                    "scan_id": scan_ids[i, sd],
                                }
                            else:
                                sub_sensor_packages[sd] = None
                    # add data package
                    data_packages.append(
                        Data_package(
                            typ="FUSEDRADAR", sensor_id=0, sensor_name="Fused", time=time[i], data_idx=i, scan_id=scan_id[i], subsensors=sub_sensor_packages
                        )
                    )

        # parse radar
        if "RADAR" in self.data2load:
            radar_data = cbor_handler.get_data().get("radar", None)
            if radar_data:
                # resolve sensor name entries in data2load
                radar_idx_to_name = [None] * len(radar_data)
                for rd in radar_data.values():
                    radar_idx_to_name[rd["internal-identifier"]] = rd["name"]
                self.resolve_str_in_data2load(radar_idx_to_name, "RADAR")
                # get data
                for rad_id, rad_name in enumerate(radar_idx_to_name):
                    if self.data2load["RADAR"] == [] or rad_id in self.data2load["RADAR"]:
                        rad_gp = radar_data[rad_name]
                        time = np.array(rad_gp["timestamps"]) * 1000
                        scan_id = rad_gp["scan_id"]
                        for i in range(len(rad_gp["timestamps"])):
                            data_packages.append(
                                Data_package(typ="RADAR", sensor_id=rad_id, sensor_name=rad_name, time=time[i], data_idx=i, scan_id=scan_id[i])
                            )

        # get sensor map
        sensor_map = {"LABEL": None}
        timestamp_map = {}
        subsensor_map = {}
        scanid_map = {}
        sensor_name_map = {}
        for dp in data_packages:
            if dp.type not in sensor_map:
                sensor_map[dp.type] = {}
                timestamp_map[dp.type] = {}
                subsensor_map[dp.type] = {}
                scanid_map[dp.type] = {}
                sensor_name_map[dp.type] = {}
            if dp.sensor_id not in sensor_map[dp.type]:
                sensor_map[dp.type][dp.sensor_id] = None
                timestamp_map[dp.type][dp.sensor_id] = None
                subsensor_map[dp.type][dp.sensor_id] = None
                scanid_map[dp.type][dp.sensor_id] = None
                sensor_name_map[dp.type][dp.sensor_id] = None

        # sort by timestamps
        data_packages.sort(key=lambda x: x.create_sort_key())

        # feed data packages to meta data list and fill aligned data
        framenumber = 0
        # last_label_location = None
        for dp in data_packages:
            # update maps
            timestamp_map[dp.type][dp.sensor_id] = dp.time
            sensor_map[dp.type][dp.sensor_id] = dp.data_idx
            subsensor_map[dp.type][dp.sensor_id] = dp.subsensors
            scanid_map[dp.type][dp.sensor_id] = dp.scan_id
            sensor_name_map[dp.type][dp.sensor_id] = dp.sensor_name
            # # create respective labelfile name
            # if 'LABEL' in self.data2load:
            #     if dp.type == 'FUSEDRADAR':
            #         sensor_map['LABEL'] = {'file': sloc + '/Labels/' + sn + '.h5', 'frame': dp.data_idx, 'source': lsrc, 'sensor': dp.sensor_name}
            #         last_label_location = sensor_map['LABEL']
            #     elif dp.type == 'LIDAR':
            #         # sensor_map['LABEL'] = last_label_location
            #         sensor_map['LABEL'] = {'file': sloc + '/Labels_Sensorwise/' + sn + '.h5', 'frame': dp.data_idx, 'source': lsrc, 'sensor': dp.sensor_name}
            #         last_label_location = sensor_map['LABEL']
            #     elif dp.type == 'CAMERA':
            #         sensor_map['LABEL'] = last_label_location
            #     elif dp.type == 'RADAR':
            #         sensor_map['LABEL'] = {'file': sloc + '/Labels_Sensorwise/' + sn + '.h5', 'frame': dp.data_idx, 'source': lsrc, 'sensor': dp.sensor_name}
            #         last_label_location = sensor_map['LABEL']

            # create meta data entries, field framenumber will be replaced afterwards
            self.file_meta_data_list.append(
                DataProviderMetaData(
                    sensor_type=dp.type,
                    sensor_id=dp.sensor_id,
                    sensor_name=dp.sensor_name,
                    source_type=styp,
                    folder=sloc,
                    sequence_name=snam,
                    sequence_number=sequence_count,
                    aligned_sensor_names=copy.deepcopy(sensor_name_map),
                    aligned_fids=copy.deepcopy(sensor_map),
                    aligned_timestamps=copy.deepcopy(timestamp_map),
                    aligned_sids=copy.deepcopy(scanid_map),
                    aligned_subsensors=copy.deepcopy(subsensor_map),
                    framenumber=framenumber,
                    data_handlers={"cbor_handler": cbor_handler},
                    flags=dict(),
                )
            )
            # increase frame counter
            framenumber += 1

        # increment sequence counter
        sequence_count += 1

        # report current sequence counter
        return sequence_count

    def resolve_str_in_data2load(self, idx_to_name: List[str], sensor_type: str):
        """
        resolve string name entries in data2load

        Args:
            idx_to_name: name lookup
            sensor_type: sensor type
        """
        # resolve sensor name entries in data2load
        sensor_lookup = {sname.decode("UTF-8") if type(sname) == bytes else sname: sid for sid, sname in enumerate(idx_to_name)}
        for i, sid in enumerate(self.data2load[sensor_type]):
            if type(sid) is str:
                if sid in sensor_lookup:
                    self.data2load[sensor_type][i] = sensor_lookup[sid]
                else:
                    print("WARNING: Sensor name %s not found in data" % sid)

    def get_current_radar_version(self) -> Optional[str]:
        """
        get current radar data version

        Returns:
            radar data version if exists, None otherwise
        """
        return self.radar_versions[self.current_radar_version_index]

    """
    set current radar version
    """

    def set_current_radar_version(self, ver: str):
        """
        set current radar data version

        Args:
            ver: radar data version
        """
        if ver in self.radar_versions:
            # set version
            self.current_radar_version_index = self.radar_versions.index(ver)
            # kill cache to prevent loading data of old version from cache
            self.clear_data_cache()
        else:
            print("WARNING: Radar version {} not available".format(ver))

    """
    toggle radar version
    """

    def toggle_current_radar_version(self):
        """
        toggle radar data version
        """
        self.set_current_radar_version(self.radar_versions[(self.current_radar_version_index + 1) % len(self.radar_versions)])

    def get_radar_versions(self) -> List[Optional[str]]:
        """
        get list of radar data versions

        Returns:
            copy of radar data versions list
        """
        return copy.deepcopy(self.radar_versions)

    def add_data_manipulator(self, dm_class: DataManipulator, dm_params: Dict[Any, Any] = dict()):
        """
        add a data manipulator

        Args:
            dm_class:   manipulator to register
            dm_params:  individual manipulator constuctor input arguments
        """
        # check input
        if not (issubclass(dm_class, DataManipulator) and type(dm_params) is dict):
            raise ValueError("Input class derived from DataManipulator and argument dict for calling constructor in DataProviderGenerator.add_data_manipulator")
        self.data_manipulators.append((dm_class, dm_params))


In [ ]:
import numpy as np


class Radar(object):

    # static variables
    pointcloud_rows = [
        "X",
        "Y",
        "elevation_deg_vcs",
        "RCS",
        "Speed",
        "doppl_propasal_0",
        "doppl_propasal_1",
        "doppl_propasal_2",
        "doppl_propasal_3",
        "doppl_propasal_4",
        "looktype",
        "isSingleTarget",
        "radar_idx",
        "timestamp",
        "fid",
        "label"
    ]

    def __init__(self, pointcloud, radar_calib, timestamp=None, frame_id=None, scan_id=None, sensor_name=None, version="v0", subsensors=None):
        self.pointcloud = pointcloud  # see Radar.pointcloud_rows
        self.radar_calib = radar_calib  # dict: key=sensor, value=RadarCalib
        self.ego_transform = np.identity(4)
        self.ego_transform_to_previous = np.identity(4)
        self.v_lon = 0
        self.v_lat = 0
        self.yawrate = 0
        self.looktype = None
        self.is_fused = subsensors is not None  # fused from multiple sensors?
        self.subsensors = subsensors  # information of sensors Radar is fused from
        self.timestamp = timestamp
        self.scan_id = scan_id
        self.frame_id = frame_id
        self.sensor_name = sensor_name
        self.version = version

    def __eq__(self, other):
        equal = np.array_equal(self.pointcloud, other.pointcloud)
        equal = equal & np.array_equal(self.ego_transform, other.ego_transform)
        equal = equal & np.array_equal(self.ego_transform_to_previous, other.ego_transform_to_previous)
        equal = equal & (self.v_lon == other.v_lon)
        equal = equal & (self.v_lat == other.v_lat)
        equal = equal & (self.yawrate == other.yawrate)
        # equal = equal & (self.radar_calib == other.radar_calib)
        equal = equal & (self.looktype == other.looktype)
        equal = equal & (self.is_fused == other.is_fused)
        equal = equal & (self.subsensors == other.subsensors)
        equal = equal & (self.timestamp == other.timestamp)
        equal = equal & (self.frame_id == other.frame_id)
        equal = equal & (self.scan_id == other.scan_id)
        equal = equal & (self.sensor_name == other.sensor_name)
        equal = equal & (self.version == other.version)

        return equal
